# Perceived Dependence of Stocks and Correlation: Monthly Fama and MacBeth Regression

For my thesis, i will look into the difference between the perceived dependence and the actual correlation of the return of stocks with the market return. In this notebook, we aim to integrate data from different sources and prepare it for Fama and MacBeth (1973) regressions as part of the robustness tests.

In [ ]:
# Warning: The full script takes +- 1,5 hours and +20 GB RAM to run.  Run on Google Collab (Pro) for faster run time.
# Note that in Google collab you have to upload the data files and notebook seperately, and the data files will be removed everytime you run.
# Always have a copy of the datafiles.

## Imports
We import the software libs we need.

We will need the following imports:

In [ ]:
# TODO Extra: Check what we need and delete the rest

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from pandas.tseries.offsets import MonthBegin, MonthEnd

# Statistical modeling and tests
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.stats.sandwich_covariance import cov_hac
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import acorr_ljungbox

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates

# File handling
import os
import openpyxl

## Reading the Data from Excel
We read in the following excels:
- *Dates.xlsx* into the *dates* dataframe with an MMYY column,
- Data about return of stocks and indexes:
    - *Stocks_DailyReturn.xlsx* into the *Stocks_DailyReturn_Dataframes['MMYY']* dataframes for each quarter,
    - *Index_DailyReturn.xlsx* into the *Index_DailyReturn_Dataframe* dataframe,
    - *Stocks_MonthlyReturn.xlsm* into the *Stocks_MonthlyReturn_Dataframes['MMYY']* dataframes for each quarter,
    - *Index_MonthlyReturn.xlsx* into the *Index_MonthlyReturn_Dataframe* dataframe,  
    - and *IndexEUR50_DailyReturn.xlsx* into the *IndexEUR50_DailyReturn_Dataframe* dataframe.
- Data about characteristics of stocks:
    - *Stocks_Turnover.xlsm* into the *Stocks_MonthlyTurnover_Dataframes["MMYY']* dataframes for each month,
    - *Stocks_OperatingProfitability.xlsm* into the *Stocks_MonthlyOpProf_Dataframes['MMYY']* dataframes for each month,
    - *Stocks_TotalAssets.xlsm* into the *Stocks_MonthlyTotalAssets_Dataframes['MMYY']* dataframes for each month,      
    - *Stocks_MarketCapitalization.xlsm* into the *Stocks_MonthlyMarketCap_Dataframes[MMYY]* dataframes for each month,     
    - *Stocks_MarketValue.xlsm* into the *Stocks_MonthlyMarketVal_Dataframes[MMYY]* dataframes for each month,                
    - *Stocks_CommonEquity.xlsm* into the *Stocks_MonthlyComEq_Dataframes[MMYY]* dataframes for each month,            
    - *Stocks_IndustryExchange.xlsm* into the *Stocks_IndustryExchange_Dataframes[MMYY]* dataframes for each month,                                                 
    - and *Stocks_TradingVolume* into the *Stocks_DailyTradingVolume[MMYY]* dataframes for each month.

### Dates
We import an excel file *Dates.xlsx* that describes certain periods for each MM YYYYY, in the form of a dataframe *dates*.

**Note that the Dates excel has been altered to include more dates, in comparison to the dates in the main results.**

Let's first read in the excel where I defined the correct time periods for every monthly data point. In this excel, i defined the month and years to be tested, and the periods that will be used for this month-year in the calculation of various variables. This will later be used to define variables: returns in these periods are used to calculate the frequency of comovement, beta and the return of stocks. Since returns are only available on weekdays, each start- and end date are adjusted in case they are on weekend.

The definitions of the variables for each MMYYYY from January 2002 to February 2023 are clarified as follows:

- **Monthly Stock Return**: This is calculated by determining the percentage difference between the stock's return on the last weekday before the start of the month (StartMonthMinus1_Weekday) and the stock's return on the last weekday of the month (EndMonth_Weekday). For example, the returns of Tuesday 31 December 2019 until Friday 31 January 2020 are used to calculate the return in January 2020.

- **Beta and Comovement**: For any given month, these metrics are calculated based on the returns from the index and the stock returns from the year preceding that month. The calculation period begins on the last weekday before the start of the previous year (StartPastYearMinus1_Weekday) and ends on the last weekday before the start of the month in question (EndPastYear_Weekday). In our example of January 2020, returns between Monday 31 December 2018 and Tuesday 31 December 2019 would be considered.

While beta is calculated using daily return differences, comovement is measured using weekly return differences. Note the **Minus 1 Day**: The day preceding the start of each period is also considered in the calculations to include the return percentage on the first day of the period.

For other variables definitions, look into the Appendix B of the paper.

In [ ]:
Dates_ExcelPath = "Dates.xlsx"
dates = pd.read_excel(Dates_ExcelPath)
dates.head(2)

,Month,Year,Index,StartMonth,StartMonth_Weekday,StartMonthMinus1,StartMonthMinus1_Weekday,EndMonth,EndMonth_Weekday,StartPastYear,...,StartPastMonthMinus1,StartPastMonthMinus1_Weekday,EndPastMonth,EndPastMonth_Weekday,StartPastYearExclPastMonth,StartPastYearExclPastMonth_Weekday,StartPastYearExclPastMonthMinus1,StartPastYearExclPastMonthMinus1_Weekday,EndPastYearExclPastMonth,EndPastYearExclPastMonth_Weekday
0,1,2002,102,2002-01-01,2002-01-01,2001-12-31,2001-12-31,2002-01-31,2002-01-31,2001-01-01,...,2001-11-30,2001-11-30,2001-12-31,2001-12-31,2001-01-01,2001-01-01,2000-12-31,2000-12-29,2001-11-30,2001-11-30
1,2,2002,102,2002-02-01,2002-02-01,2002-01-31,2002-01-31,2002-02-28,2002-02-28,2001-02-01,...,2001-12-31,2001-12-31,2002-01-31,2002-01-31,2001-02-01,2001-02-01,2001-01-31,2001-01-31,2001-12-31,2001-12-31


In [ ]:
# Step 1: Convert 'Index' to string and prepend zeros if less than 4 characters.
dates['Index'] = dates['Index'].apply(lambda x: f'{x:04}')
dates.head(2)

,Month,Year,Index,StartMonth,StartMonth_Weekday,StartMonthMinus1,StartMonthMinus1_Weekday,EndMonth,EndMonth_Weekday,StartPastYear,...,StartPastMonthMinus1,StartPastMonthMinus1_Weekday,EndPastMonth,EndPastMonth_Weekday,StartPastYearExclPastMonth,StartPastYearExclPastMonth_Weekday,StartPastYearExclPastMonthMinus1,StartPastYearExclPastMonthMinus1_Weekday,EndPastYearExclPastMonth,EndPastYearExclPastMonth_Weekday
0,1,2002,0102,2002-01-01,2002-01-01,2001-12-31,2001-12-31,2002-01-31,2002-01-31,2001-01-01,...,2001-11-30,2001-11-30,2001-12-31,2001-12-31,2001-01-01,2001-01-01,2000-12-31,2000-12-29,2001-11-30,2001-11-30
1,2,2002,0102,2002-02-01,2002-02-01,2002-01-31,2002-01-31,2002-02-28,2002-02-28,2001-02-01,...,2001-12-31,2001-12-31,2002-01-31,2002-01-31,2001-02-01,2001-02-01,2001-01-31,2001-01-31,2001-12-31,2001-12-31


### Returns

#### Daily Stock Returns
We import an excel file *Stocks_DailyReturn.xlsx* that holds data from DataStream about the stocks in the index in each quarter (one quarter per sheet) and their daily return index. We save them into a dictionairy *Stocks_DailyReturn_Dataframes* that holds a dataframe MYYY for each quarter and consists out of the stock names in the index column, the dates (starting from the last weekday before the year prior, until the last weekday of the quarter) as columns and the return index as values.

For each quarter, we have used the dates to construct the following prompt in DataStream:

     DSGRID("LDJSTOXXMMYY";"X(RI)~E";"{start_date}"; "{end_date}";"D";"RowHeader=true;TimeSeriesList=true;ColHeader=true;Transpose=true;Curn=true;DispSeriesDescription=false;YearlyTSFormat=false;QuarterlyTSFormat=false;MonthlyTSFormat=False";"")'
     
This is used to save the daily returns of each month and the year prior to this month in an excel called 'Stocks_Return'. For the Index, the code DJSTOXX is used. The ~E after the return index refers to prices that are all in euro.

We will first read in the Excel file where the daily returns of the stocks are saved. These are loaded in DataStream and are not modified. The formulas are generated with the other notebook.  The *Stocks_DailyReturn_Dataframes['MMYY']*  are used to save these dataframes.

In [ ]:
#Note that these are actually per quarter instead of per month
# since stocks in the index doesnt change in a quarter so easier to take them together, less prompts

# Path to the Excel file
file_path = 'Stocks_DailyReturn.xlsx'

# Initialize an empty dictionary to store dataframes
Stocks_DailyReturn_Dataframes = {}

# Read the unique index values from the 'dates' dataframe
unique_indexes = dates['Index'].unique()

# Loop through each unique index value
for index_value in unique_indexes:
    # Construct the sheet name from the index value
    sheet_name = str(index_value)
    try:
        # Attempt to read the sheet into a dataframe
        df = pd.read_excel(file_path, sheet_name=sheet_name, index_col=0)
        # Store the dataframe in the dictionary with the sheet name as the key
        Stocks_DailyReturn_Dataframes[sheet_name] = df
    except Exception as e:
        # If there's an error reading the sheet, print the error message
        print(f"Error reading sheet {sheet_name}: {e}")

In [ ]:
# At this point, Stocks_DailyReturn_Dataframes will contain all the dataframes keyed by their respective sheet names.
Stocks_DailyReturn_Dataframes['0102'].head(2)

,CURRENCY,2000-12-29 00:00:00,2001-01-01 00:00:00,2001-01-02 00:00:00,2001-01-03 00:00:00,2001-01-04 00:00:00,2001-01-05 00:00:00,2001-01-08 00:00:00,2001-01-09 00:00:00,2001-01-10 00:00:00,...,2002-03-18 00:00:00,2002-03-19 00:00:00,2002-03-20 00:00:00,2002-03-21 00:00:00,2002-03-22 00:00:00,2002-03-25 00:00:00,2002-03-26 00:00:00,2002-03-27 00:00:00,2002-03-28 00:00:00,2002-03-29 00:00:00
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,E,823.58,823.58,795.30,758.38,829.48,856.33,844.27,841.79,828.14,...,517.59,527.79,527.27,518.79,519.18,518.21,517.33,520.69,544.36,544.36
ABB LTD N - TOT RETURN IND,E,1957.55,1957.55,1955.18,1849.66,1925.13,1939.00,1965.14,2007.85,1982.15,...,685.70,677.32,663.43,630.43,616.89,621.83,579.15,622.14,642.96,642.96


#### Daily Index Returns
We import an excel file *Index_DailyReturn.xlsx* that holds data from DataStream about the daily return index of the EUR600, starting from 29/12/2000 until 29/3/2024. It is saved into a dataframe *Index_DailyReturn_Dataframe*, with a column *Date*  and a column *Return Index*.

As explained above, an excel file was also made with the help of datastream to save the daily return index of the EUR600. These are saved in a dataframe called *Index_DailyReturn_Dataframe*.

In [ ]:
# The path to the Excel file
Index_DailyReturn_ExcelPath = 'Index_DailyReturn.xlsx'

# Read the specified sheet into a DataFrame
Index_DailyReturn_Dataframe = pd.read_excel(Index_DailyReturn_ExcelPath)
Index_DailyReturn_Dataframe.head(2)

,Date,Return Index
0,2000-12-29,100.0
1,2001-01-01,100.0


#### Monthly Stock Returns
We import an excel file *Stocks_MonthlyReturn.xlsm* that holds data from DataStream about the stocks in the index in each quarter (one quarter per sheet) and their daily return index. We save them into a dictionairy *Stocks_MonthlyReturn_Dataframes* that holds a dataframe MYYY for each quarter and consists out of the stock names in the index column, the dates (starting from the last weekday before the year prior, until the last weekday of the quarter) as columns and the return index as values.

For each quarter, we have used the dates to construct the following prompt in DataStream:

     = DSGRID.(“LDJSTOXXMMYY”, “X(RI)~E”, “startDate”, “endDate”, “M”,  “RowHeader=true;TimeSeriesList=true;ColHeader=true;Transpose=true;DispSeriesDescription=false;YearlyTSFormat=false;QuarterlyTSFormat=false;Clearself=y;MonthlyTSFormat=True”)
     
This is used to save the monthly returns of each quarter and the three year prior to this quarter in an excel called 'Stocks_MonthlyReturn'. For the Index, the code DJSTOXX is used. The ~E after the return index refers to prices that are all in euro.

In [ ]:
#Note that these are actually per quarter instead of per month
# since stocks in the index doesnt change in a quarter so easier to take them together, less prompts

# Path to the Excel file
file_path = 'Stocks_MonthlyReturn.xlsm'

# Initialize an empty dictionary to store dataframes
Stocks_MonthlyReturn_Dataframes = {}

# Read the unique index values from the 'dates' dataframe
unique_indexes = dates['Index'].unique()

# Loop through each unique index value
for index_value in unique_indexes:
    # Construct the sheet name from the index value
    sheet_name = str(index_value)
    try:
        # Attempt to read the sheet into a dataframe
        df = pd.read_excel(file_path, sheet_name=sheet_name, index_col=0)
        # Store the dataframe in the dictionary with the sheet name as the key
        Stocks_MonthlyReturn_Dataframes[sheet_name] = df
    except Exception as e:
        # If there's an error reading the sheet, print the error message
        print(f"Error reading sheet {sheet_name}: {e}")

In [ ]:
# At this point, Stocks_MonthlyReturn_Dataframes will contain all the dataframes keyed by their respective sheet names.
Stocks_MonthlyReturn_Dataframes['0102'].head(2)

,1999-01-01,1999-02-01,1999-03-01,1999-04-01,1999-05-01,1999-06-01,1999-07-01,1999-08-01,1999-09-01,1999-10-01,...,2001-06-01,2001-07-01,2001-08-01,2001-09-01,2001-10-01,2001-11-01,2001-12-01,2002-01-01,2002-02-01,2002-03-01
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,334.69,369.11,360.81,379.41,417.90,437.52,463.67,497.32,516.88,477.90,...,859.70,786.10,664.19,608.49,456.85,491.28,574.71,595.39,531.58,479.07
ABB LTD N - TOT RETURN IND,1032.11,1021.68,1130.20,1269.54,1450.67,1366.47,1540.00,1549.39,1631.72,1647.76,...,1491.66,1270.67,912.99,761.84,524.29,681.53,796.16,764.85,659.15,631.88


#### Monthly Index Returns
We import an excel file *Index_MonthlyReturn.xlsm* that holds data from DataStream about the monthly return index of the EUR600, starting from 29/12/2000 until 29/3/2024. It is saved into a dataframe *Index_MonthlyReturn_Dataframe*, with a column *Date*  and a column *Return Index*.

In [ ]:
# The path to the Excel file
Index_MonthlyReturn_ExcelPath = 'Index_MonthlyReturn.xlsx'

# Read the specified sheet into a DataFrame
Index_MonthlyReturn_Dataframe = pd.read_excel(Index_MonthlyReturn_ExcelPath)
Index_MonthlyReturn_Dataframe.head(2)

,Date,Return Index
0,1999-01-01,NaN
1,1999-02-01,NaN


#### Daily EUR50 Index Returns
We import an excel file *IndexEUR50_DailyReturn.xlsx* that holds data from DataStream about the daily return index of the EUR 50, starting from 29/12/2000 until 29/3/2024. It is saved into a dataframe *IndexEUR50_DailyReturn_Dataframe*, with a column *Date*  and a column *Return Index*.

In [ ]:
# The path to the Excel file
IndexEUR50_DailyReturn_ExcelPath = 'IndexEUR50_DailyReturn.xlsx'

# Read the specified sheet into a DataFrame
IndexEUR50_DailyReturn_Dataframe = pd.read_excel(IndexEUR50_DailyReturn_ExcelPath)
IndexEUR50_DailyReturn_Dataframe.head(2)

,Date,Return Index
0,2000-12-01,NaN
1,2000-12-04,NaN


### Stock Characteristics

#### Monthly Turnover
We import an excel file *Stocks_Turnover.xlsm* that holds data from DataStream about the stocks in the index in each quarter (one quarter per sheet) and their monthly turnover by volume. We save them into a dictionairy *Stocks_MonthlyTurnover_Dataframes* that holds a dataframe MMYY for each month and consists out of the stock names in the index column, the dates (starting from the first day of the 2 months prior to MMYY, until the first day of the month prior) as columns and the monthly turnover as values.

For each month, we have used the dates to construct the following prompt in DataStream:

     = DSGRID.(“LDJSTOXXMMYY”, “VO”, “startDate”, “endDate”, “M”,  "RowHeader=true;TimeSeriesList=true;ColHeader=true;Transpose=true;DispSeriesDescription=false;YearlyTSFormat=false;QuarterlyTSFormat=false;Clearself=y;MonthlyTSFormat=True”)
     
For the Index, the code DJSTOXX is used.

In [ ]:
# Function to convert MMYY to corresponding Excel sheet name based on the quarter
def get_sheet_name(mmyy):
    month = int(mmyy[:2])
    year = mmyy[2:]
    quarter = (month - 1) // 3 + 1
    quarter_start_month = (quarter - 1) * 3 + 1
    return f'{quarter_start_month:02}{year}'

In [ ]:
# Open the Excel workbook
wb = openpyxl.load_workbook('Stocks_Turnover.xlsm', data_only=True)

# Dictionary to hold the monthly dataframes
Stocks_MonthlyTurnover_Dataframes = {}

# Process each month in the testing period
for index, row in dates.iterrows():
    month = row['Month']
    year = str(row['Year'])[2:]
    mmyy =  f"{month:02d}{year}"
    sheet_name = get_sheet_name(mmyy)
    ws = wb[sheet_name]

    # Create dataframe from the excel sheet
    data = ws.values
    headers = next(data)[1:]  # Skip the first column (stock names) for headers
    # Format headers as date strings for easy matching
    cols = [d.strftime('%d/%m/%Y') if isinstance(d, datetime) else d for d in headers]
    data = list(data)
    index = [r[0] for r in data]  # First column as index (stock names)
    data = (r[1:] for r in data)  # Remaining data

    df = pd.DataFrame(data, index=index, columns=cols)

    # Convert the columns from 'thousands' to actual values
    df *= 1000

    # Get the two relevant dates
    start_past_month = (pd.to_datetime(row['StartPastMonth'])).strftime('%d/%m/%Y')
    two_months_prior = (pd.to_datetime(row['StartPastMonth']) - pd.DateOffset(months=1)).strftime('%d/%m/%Y')
    # Filter the DataFrame to keep only the required columns, if they exist
    required_dates = [two_months_prior, start_past_month]
    available_dates = [date for date in required_dates if date in df.columns]
    if not available_dates:
        print(f"Required dates {required_dates} are missing in the sheet {sheet_name} for {mmyy}.")
        continue

    filtered_df = df[available_dates]

    if filtered_df.empty:
        print(f"No data available for {mmyy} in columns {available_dates}")
        continue

    # Add to dictionary
    Stocks_MonthlyTurnover_Dataframes[mmyy] = filtered_df
    # Change headers back to YYYY-MM-DD format
    filtered_df.columns = [pd.to_datetime(date).strftime('%Y-%m-%d') for date in filtered_df.columns]

# Close the workbook
wb.close()

In [ ]:
# At this point, Stocks_DailyReturn_Dataframes will contain all the dataframes keyed by their respective sheet names.
Stocks_MonthlyTurnover_Dataframes['0102'].head(2)

,2001-01-11,2001-01-12
3I GROUP - TURNOVER BY VOLUME,134771300.0,111941100.0
ABB LTD N - TURNOVER BY VOLUME,242604200.0,164805600.0


#### Monthly Operating Profitability
We import an excel file *Stocks_OperatingProfitability.xlsm* that holds data from DataStream about the stocks in the index in each quarter (one quarter per sheet) and their monthly operating profitability. We save them into a dictionairy *Stocks_MonthlyOpProf_Dataframes* that holds a dataframe MMYY for each month and consists out of the stock names in the index column, the dates (starting from the first day of the year prior to MMYY, until the first day of the month prior) as columns and the monthly operting income as values.

For each month, we have used the dates to construct the following prompt in DataStream:

     = DSGRID.(“LDJSTOXXMMYY”, “X(WC01250)~E”, “startDate”, “endDate”, “M”,  "RowHeader=true;TimeSeriesList=true;ColHeader=true;Transpose=true;DispSeriesDescription=false;YearlyTSFormat=false;QuarterlyTSFormat=false;Clearself=y;MonthlyTSFormat=True”)
     
For the Index, the code DJSTOXX is used.

In [ ]:
# Open the Excel workbook
wb = openpyxl.load_workbook('Stocks_OperatingProfitability.xlsm', data_only=True)

# Dictionary to hold the monthly dataframes
Stocks_MonthlyOpProf_Dataframes = {}

# Process each month in the testing period
for index, row in dates.iterrows():
    month = row['Month']
    year = str(row['Year'])[2:]
    mmyy = f"{month:02d}{year}"
    sheet_name = get_sheet_name(mmyy)
    ws = wb[sheet_name]

    # Create dataframe from the excel sheet
    data = ws.values
    headers = next(data)[1:]  # Skip the first column (stock names) for headers
    # Format headers as date strings for easy matching
    cols = [d.strftime('%d/%m/%Y') if isinstance(d, datetime) else d for d in headers]
    data = list(data)
    index = [r[0] for r in data]  # First column as index (stock names)
    data = (r[1:] for r in data)  # Remaining data

    df = pd.DataFrame(data, index=index, columns=cols)
    df.columns = pd.to_datetime(df.columns)

    # Get the two relevant dates
    start_past_year = pd.to_datetime(row['StartPastYear']).strftime('%d/%m/%Y')
    start_past_month = pd.to_datetime(row['StartPastMonth']).strftime('%d/%m/%Y')

    filtered_df = df.loc[:, start_past_year:start_past_month]

    # Add to dictionary
    Stocks_MonthlyOpProf_Dataframes[mmyy] = filtered_df
    # Change headers back to YYYY-MM-DD format for consistency
    filtered_df.columns = [date.strftime('%Y-%m-%d') for date in filtered_df.columns]

# Close the workbook
wb.close()

In [ ]:
# At this point, Stocks_DailyReturn_Dataframes will contain all the dataframes keyed by their respective sheet names.
Stocks_MonthlyOpProf_Dataframes['0102'].head(2)

,2001-01-01,2001-01-02,2001-01-03,2001-01-04,2001-01-05,2001-01-06,2001-01-07,2001-01-08,2001-01-09,2001-01-10,2001-01-11,2001-01-12
3I GROUP PLC - OPERATING INCOME,194113,191824,190864,175608,175185,182213,182305,177582,174596,175948,175863,173899
ABB LTD - OPERATING INCOME,566514,562226,560341,564676,559018,566740,566623,571023,568283,581300,585347,584941


#### Monthly Total Assets
We import an excel file *Stocks_TotalAssets.xlsm* that holds data from DataStream about the stocks in the index in each quarter (one quarter per sheet) and their monthly total assets. We save them into a dictionairy *Stocks_MonthlyTotalAssets_Dataframes* that holds a dataframe MMYY for each month and consists out of the stock names in the index column, the dates (starting from the first day of the month that occurs before the year prior to MMYY, until the first day of the month prior) as columns and the monthly turnover as values.

For each month, we have used the dates to construct the following prompt in DataStream:

     = DSGRID.(“LDJSTOXXMMYY”, “X(WC02999)~E”, “startDate”, “endDate”, “M”,  "RowHeader=true;TimeSeriesList=true;ColHeader=true;Transpose=true;DispSeriesDescription=false;YearlyTSFormat=false;QuarterlyTSFormat=false;Clearself=y;MonthlyTSFormat=True”)
     
For the Index, the code DJSTOXX is used.

In [ ]:
# Open the Excel workbook
wb = openpyxl.load_workbook('Stocks_TotalAssets.xlsm', data_only=True)

# Dictionary to hold the monthly dataframes
Stocks_MonthlyTotalAssets_Dataframes = {}

# Process each month in the testing period
for index, row in dates.iterrows():
    month = row['Month']
    year = str(row['Year'])[2:]
    mmyy = f"{month:02d}{year}"
    sheet_name = get_sheet_name(mmyy)
    ws = wb[sheet_name]

    # Create dataframe from the excel sheet
    data = ws.values
    headers = next(data)[1:]  # Skip the first column (stock names) for headers
    # Format headers as date strings for easy matching
    cols = [d.strftime('%d/%m/%Y') if isinstance(d, datetime) else d for d in headers]
    data = list(data)
    index = [r[0] for r in data]  # First column as index (stock names)
    data = (r[1:] for r in data)  # Remaining data

    df = pd.DataFrame(data, index=index, columns=cols)
    df.columns = pd.to_datetime(df.columns)

    # Get the two relevant dates
    start_past_year_minus1 = (pd.to_datetime(row['StartPastYear']) - pd.DateOffset(months=1)).strftime('%d/%m/%Y')
    start_past_month = pd.to_datetime(row['StartPastMonth']).strftime('%d/%m/%Y')

    filtered_df = df.loc[:, start_past_year_minus1:start_past_month]

    # Add to dictionary
    Stocks_MonthlyTotalAssets_Dataframes[mmyy] = filtered_df
    # Change headers back to YYYY-MM-DD format for consistency
    filtered_df.columns = [date.strftime('%Y-%m-%d') for date in filtered_df.columns]

# Close the workbook
wb.close()

In [ ]:
# At this point, Stocks_DailyReturn_Dataframes will contain all the dataframes keyed by their respective sheet names.
Stocks_MonthlyTotalAssets_Dataframes['0102'].head(2)

,2000-01-12,2001-01-01,2001-01-02,2001-01-03,2001-01-04,2001-01-05,2001-01-06,2001-01-07,2001-01-08,2001-01-09,2001-01-10,2001-01-11,2001-01-12
3I GROUP PLC - TOTAL ASSETS,12113208,11747017,11608491,11550376,9793782,9770171,10162154,10167253,9903878,9737307,9812753,9808003,9698469
ABB LTD - TOTAL ASSETS,32466838,35117018,34851189,34734352,35003048,34652334,35131012,35123763,35396530,35226661,36033568,36284452,36259279


#### Monthly Market Capitalization
We import an excel file *Stocks_MarketCapitalization.xlsm* that holds data from DataStream about the stocks in the index in each quarter (one quarter per sheet) and their monthly market capitalization. We save them into a dictionairy *Stocks_MonthlyMarketCap_Dataframes* that holds a dataframe MMYY for each month and consists out of the stock names in the index column, the dates (starting from the first day of the year prior to MMYY, until the first day of the month prior) as columns and the monthly market capitalization as values.

For each month, we have used the dates to construct the following prompt in DataStream:

     = DSGRID.(“LDJSTOXXMMYY”, “X(WC08001)~E”, “startDate”, “endDate”, “M”,  "RowHeader=true;TimeSeriesList=true;ColHeader=true;Transpose=true;DispSeriesDescription=false;YearlyTSFormat=false;QuarterlyTSFormat=false;Clearself=y;MonthlyTSFormat=True”)
     
For the Index, the code DJSTOXX is used.

In [ ]:
# Open the Excel workbook
wb = openpyxl.load_workbook('Stocks_MarketCapitalization.xlsm', data_only=True)

# Dictionary to hold the monthly dataframes
Stocks_MonthlyMarketCap_Dataframes = {}

# Process each month in the testing period
for index, row in dates.iterrows():
    month = row['Month']
    year = str(row['Year'])[2:]
    mmyy = f"{month:02d}{year}"
    sheet_name = get_sheet_name(mmyy)
    ws = wb[sheet_name]

    # Create dataframe from the excel sheet
    data = ws.values
    headers = next(data)[1:]  # Skip the first column (stock names) for headers
    # Format headers as date strings for easy matching
    cols = [d.strftime('%d/%m/%Y') if isinstance(d, datetime) else d for d in headers]
    data = list(data)
    index = [r[0] for r in data]  # First column as index (stock names)
    data = (r[1:] for r in data)  # Remaining data

    df = pd.DataFrame(data, index=index, columns=cols)
    df.columns = pd.to_datetime(df.columns)

    # Get the two relevant dates
    start_past_year = pd.to_datetime(row['StartPastYear']).strftime('%d/%m/%Y')
    start_past_month = pd.to_datetime(row['StartPastMonth']).strftime('%d/%m/%Y')

    filtered_df = df.loc[:, start_past_year:start_past_month]

    # Add to dictionary
    Stocks_MonthlyMarketCap_Dataframes[mmyy] = filtered_df
    # Change headers back to YYYY-MM-DD format for consistency
    filtered_df.columns = [date.strftime('%Y-%m-%d') for date in filtered_df.columns]

# Close the workbook
wb.close()

In [ ]:
# At this point, Stocks_DailyReturn_Dataframes will contain all the dataframes keyed by their respective sheet names.
Stocks_MonthlyMarketCap_Dataframes['0102'].head(2)

,2001-01-01,2001-01-02,2001-01-03,2001-01-04,2001-01-05,2001-01-06,2001-01-07,2001-01-08,2001-01-09,2001-01-10,2001-01-11,2001-01-12
3I GROUP PLC - MARKET CAPITALIZATION,10662154,10536421,10483674,7606101,7587764,7892188,7896148,7691604,7562241,7620834,7617146,7532079
ABB LTD - MARKET CAPITALIZATION,11706344,11617729,11578781,11668352,11551440,11711009,11708592,11799520,11742894,12011878,12095511,12087119


#### Monthly Market Value
We import an excel file *Stocks_MarketValue.xlsm* that holds data from DataStream about the stocks in the index in each quarter (one quarter per sheet) and their monthly market value. We save them into a dictionairy *Stocks_MonthlyMarketVal_Dataframes* that holds a dataframe MMYY for each month and consists out of the stock names in the index column, the dates (starting from the first day of the year prior to MMYY, until the first day of the month prior) as columns and the monthly market value as values.

For each month, we have used the dates to construct the following prompt in DataStream:

     = DSGRID.(“LDJSTOXXMMYY”, “X(MVC)~E”, “startDate”, “endDate”, “M”,  "RowHeader=true;TimeSeriesList=true;ColHeader=true;Transpose=true;DispSeriesDescription=false;YearlyTSFormat=false;QuarterlyTSFormat=false;Clearself=y;MonthlyTSFormat=True”)
     
For the Index, the code DJSTOXX is used.

In [ ]:
# Open the Excel workbook
wb = openpyxl.load_workbook('Stocks_MarketValue.xlsm', data_only=True)

# Dictionary to hold the monthly dataframes
Stocks_MonthlyMarketVal_Dataframes = {}

# Process each month in the testing period
for index, row in dates.iterrows():
    month = row['Month']
    year = str(row['Year'])[2:]
    mmyy = f"{month:02d}{year}"
    sheet_name = get_sheet_name(mmyy)
    ws = wb[sheet_name]

    # Create dataframe from the excel sheet
    data = ws.values
    headers = next(data)[1:]  # Skip the first column (stock names) for headers
    # Format headers as date strings for easy matching
    cols = [d.strftime('%d/%m/%Y') if isinstance(d, datetime) else d for d in headers]
    data = list(data)
    index = [r[0] for r in data]  # First column as index (stock names)
    data = (r[1:] for r in data)  # Remaining data

    df = pd.DataFrame(data, index=index, columns=cols)
    df.columns = pd.to_datetime(df.columns)

    # Get the two relevant dates
    start_past_year = pd.to_datetime(row['StartPastYear']).strftime('%d/%m/%Y')
    start_past_month = pd.to_datetime(row['StartPastMonth']).strftime('%d/%m/%Y')

    filtered_df = df.loc[:, start_past_year:start_past_month]

    # Add to dictionary
    Stocks_MonthlyMarketVal_Dataframes[mmyy] = filtered_df
    # Change headers back to YYYY-MM-DD format for consistency
    filtered_df.columns = [date.strftime('%Y-%m-%d') for date in filtered_df.columns]

# Close the workbook
wb.close()

In [ ]:
# At this point, Stocks_DailyReturn_Dataframes will contain all the dataframes keyed by their respective sheet names.
Stocks_MonthlyMarketVal_Dataframes['0102'].head(2)

,2001-01-01,2001-01-02,2001-01-03,2001-01-04,2001-01-05,2001-01-06,2001-01-07,2001-01-08,2001-01-09,2001-01-10,2001-01-11,2001-01-12
3I GROUP - MARKET VAL BY CO.,11936.8,13577.83,12101.66,11312.97,12154.57,12489.91,11337.98,9579.63,8776.34,6589.13,7085.77,8237.4
ABB LTD N - MARKET VAL BY CO.,34064,31359.98,26427.66,20833.97,24322.2,25328.88,21576.4,15503.01,12936.29,8902.68,11572.56,13519.1


#### Monthly Common Equity
We import an excel file *Stocks_CommonEquity.xlsm* that holds data from DataStream about the stocks in the index in each quarter (one quarter per sheet) and their monthly common equity. We save them into a dictionairy *Stocks_MonthlyComEq_Dataframes* that holds a dataframe MMYY for each month and consists out of the stock names in the index column, the dates (starting from the first day of the year prior to MMYY, until the first day of the month prior) as columns and the monthly common equity as values.

For each month, we have used the dates to construct the following prompt in DataStream:

     = DSGRID.(“LDJSTOXXMMYY”, “X(WC03501)~E”, “startDate”, “endDate”, “M”,  "RowHeader=true;TimeSeriesList=true;ColHeader=true;Transpose=true;DispSeriesDescription=false;YearlyTSFormat=false;QuarterlyTSFormat=false;Clearself=y;MonthlyTSFormat=True”)
     
For the Index, the code DJSTOXX is used.

In [ ]:
# Open the Excel workbook
wb = openpyxl.load_workbook('Stocks_CommonEquity.xlsm', data_only=True)

# Dictionary to hold the monthly dataframes
Stocks_MonthlyComEq_Dataframes = {}

# Process each month in the testing period
for index, row in dates.iterrows():
    month = row['Month']
    year = str(row['Year'])[2:]
    mmyy = f"{month:02d}{year}"
    sheet_name = get_sheet_name(mmyy)
    ws = wb[sheet_name]

    # Create dataframe from the excel sheet
    data = ws.values
    headers = next(data)[1:]  # Skip the first column (stock names) for headers
    # Format headers as date strings for easy matching
    cols = [d.strftime('%d/%m/%Y') if isinstance(d, datetime) else d for d in headers]
    data = list(data)
    index = [r[0] for r in data]  # First column as index (stock names)
    data = (r[1:] for r in data)  # Remaining data

    df = pd.DataFrame(data, index=index, columns=cols)
    df.columns = pd.to_datetime(df.columns)

    # Get the two relevant dates
    start_past_year = pd.to_datetime(row['StartPastYear']).strftime('%d/%m/%Y')
    start_past_month = pd.to_datetime(row['StartPastMonth']).strftime('%d/%m/%Y')

    filtered_df = df.loc[:, start_past_year:start_past_month]

    # Add to dictionary
    Stocks_MonthlyComEq_Dataframes[mmyy] = filtered_df
    # Change headers back to YYYY-MM-DD format for consistency
    filtered_df.columns = [date.strftime('%Y-%m-%d') for date in filtered_df.columns]

# Close the workbook
wb.close()

In [ ]:
# At this point, Stocks_DailyReturn_Dataframes will contain all the dataframes keyed by their respective sheet names.
Stocks_MonthlyComEq_Dataframes['0102'].head(2)

,2001-01-01,2001-01-02,2001-01-03,2001-01-04,2001-01-05,2001-01-06,2001-01-07,2001-01-08,2001-01-09,2001-01-10,2001-01-11,2001-01-12
3I GROUP PLC - COMMON SHAREHOLDERS' EQUITY,7823389,7731132,7692428,6268729,6253617,6504514,6507778,6339199,6232581,6280872,6277832,6207722
ABB LTD - COMMON SHAREHOLDERS' EQUITY,2223938,2207103,2199704,2216720,2194510,2224824,2224365,2241639,2230882,2281982,2297871,2296277


#### Industry and Exchange
We import an excel file *Stocks_IndustryExchange.xlsm* that holds data from DataStream about the stocks in the index in each quarter (one quarter per sheet) and their industry and exchange. We save them into a dictionairy *Stocks_IndustryExchange_Dataframes* that holds a dataframe MMYY for each month and consists out of the stock names in the index column and  'Industry' and 'Exchange' as columns with their corresponding values.

For each month, we constructed the following prompt in DataStream:

     = DSGRID.(“LDJSTOXXMMYY”, “NAME;INDM;EXDSCD”, “startDate”, “endDate”, “M”,  "RowHeader=true;TimeSeriesList=true;ColHeader=true;Transpose=true;DispSeriesDescription=false;YearlyTSFormat=false;QuarterlyTSFormat=false;Clearself=y;MonthlyTSFormat=True”)
     
For the Index, the code DJSTOXX is used.

In [ ]:
# Open the Excel workbook
wb = openpyxl.load_workbook('Stocks_IndustryExchange.xlsm', data_only=True)

# Dictionary to hold the monthly dataframes
Stocks_IndustryExchange_Dataframes = {}

# Process each month in the testing period
for index, row in dates.iterrows():
    month = row['Month']
    year = str(row['Year'])[2:]
    mmyy = f"{month:02d}{year}"
    sheet_name = get_sheet_name(mmyy)
    ws = wb[sheet_name]

    # Create dataframe from the excel sheet
    data = ws.values
    df = pd.DataFrame(data)

    # Keep only the relevant three rows (which will become Index, Industry, and Exchange)
    df = df.iloc[1:4, :]

    # Transpose the DataFrame
    df_t = df.T

    # Set the second row as the new index
    df_t.set_index(1, inplace=True)

    # Rename the third and fourth rows to 'Industry' and 'Exchange'
    df_t.columns = ['Industry', 'Exchange']  # This assumes only two rows are left for columns

    # Add to dictionary
    Stocks_IndustryExchange_Dataframes[mmyy] = df_t

# Close the workbook
wb.close()

In [ ]:
for key, df in Stocks_IndustryExchange_Dataframes.items(): #TODO dit hierboven zetten
    df = df.drop(df.index[0])
    Stocks_IndustryExchange_Dataframes[key] =df

In [ ]:
Stocks_IndustryExchange_Dataframes["0102"]

,Industry,Exchange
1,,
3I GROUP,Investment Trusts,LN
ABB LTD N,Electrical Components,ZU
ABBEY NATIONAL DEAD - T/O SEE 702853,Banks,LN
ABN AMRO HOLDING DEAD - 25/04/08,Banks,AM
ACCOR,Hotels & Motels,PR
...,...,...
XANSA DEAD - EX.INTO 672558,Computer Services,LN
ZARDOYA OTIS DEAD - DELIST.09/05/22,Machinery: Industrial,MC
PHARMA MAR,Pharmaceuticals,MC


Let's make a list of all the unique industries and exchanges there are across all the dataframes.

In [ ]:
# Lists to hold all the values from each column across all DataFrames
all_industries = []
all_exchanges = []

# Iterate through each DataFrame in the dictionary
for df in Stocks_IndustryExchange_Dataframes.values():
    all_industries.extend(df['Industry'].tolist())
    all_exchanges.extend(df['Exchange'].tolist())

# Getting unique values for each column
unique_industries = set(all_industries)
unique_exchanges = set(all_exchanges)

# Finding the ten most common values in each column
most_common_industries = Counter(all_industries).most_common(10)
most_common_exchanges = Counter(all_exchanges).most_common(10)

# Printing the results
print("Ten Most Common Industries:", most_common_industries)
print("Ten Most Common Exchanges:", most_common_exchanges)

Ten Most Common Industries: [('Banks', 13978), ('Telecom. Services', 5966), ('Pharmaceuticals', 4350), ('Asset Mngr, Custodian', 3887), ('Food Products', 3436), ('Food Retail & Wsale', 3297), ('Life Insurance', 3071), ('Conv. Electricity', 2971), ('Construction', 2841), ('Full Line Insurance', 2665)]
Ten Most Common Exchanges: [('LN', 45852), ('PR', 21399), ('IB', 17197), ('ZU', 12275), ('ST', 11166), ('ML', 9448), ('MC', 8250), ('AM', 8071), ('CP', 4967), ('HL', 4605)]


#### Daily Trading Volume
We import an excel file *Stocks_TradingActivity.xlsm* that holds data from DataStream about the stocks in the index in each quarter (one quarter per sheet) and their daily turnover by value. We save them into a dictionairy *Stocks_DailyTradingVolume_Dataframes* that holds a dataframe MMYY for each month and consists out of the stock names in the index column, the dates (starting from the first weekday of the year prior to MMYY, until the last weekday of the month prior) as columns and the daily turnover as values.

For each month, we have used the dates to construct the following prompt in DataStream:

     = DSGRID.(“LDJSTOXXMMYY”, “X(VA)~E”, “startDate”, “endDate”, “D”,  "RowHeader=true;TimeSeriesList=true;ColHeader=true;Transpose=true;DispSeriesDescription=false;YearlyTSFormat=false;QuarterlyTSFormat=false;Clearself=y;MonthlyTSFormat=True”)
     
For the Index, the code DJSTOXX is used.

In [ ]:
# Open the Excel workbook
wb = openpyxl.load_workbook('Stocks_TradingVolume.xlsm', data_only=True)

# Dictionary to hold the monthly dataframes
Stocks_DailyTradingVolume_Dataframes = {}

# Process each month in the testing period
for index, row in dates.iterrows():
    month = row['Month']
    year = str(row['Year'])[2:]
    mmyy =  f"{month:02d}{year}"
    sheet_name = get_sheet_name(mmyy)
    ws = wb[sheet_name]

    # Create dataframe from the excel sheet
    data = ws.values
    headers = next(data)[1:]  # Skip the first column (stock names) for headers
    # Format headers as date strings for easy matching
    cols = [d.strftime('%d/%m/%Y') if isinstance(d, datetime) else d for d in headers]
    data = list(data)
    index = [r[0] for r in data]  # First column as index (stock names)
    data = (r[1:] for r in data)  # Remaining data

    df = pd.DataFrame(data, index=index, columns=cols)

    # Get the two relevant dates - Be sure to take into account weekdays since we are  now working with dialy frequency
    # For months, if for example 1 Jan 2021 is a sunday, the monthly aggregate will still have the date of 1-1-21
    # For days, this date will not exist in the data
    start_date = (pd.to_datetime(row['StartPastYear_Weekday'])).strftime('%d/%m/%Y')
    end_date = (pd.to_datetime(row['EndPastMonth_Weekday'])).strftime('%d/%m/%Y')
    # Filter the DataFrame to keep only the required columns, if they exist
    required_dates = [start_date, end_date]
    available_dates = [date for date in required_dates if date in df.columns]
    if not available_dates:
        print(f"Required dates {required_dates} are missing in the sheet {sheet_name} for {mmyy}.")
        continue

    filtered_df = df.loc[:, start_date:end_date]

    if filtered_df.empty:
        print(f"No data available for {mmyy} in columns {available_dates}")
        continue

    # Add to dictionary
    Stocks_DailyTradingVolume_Dataframes[mmyy] = filtered_df

# Close the workbook
wb.close()

In [ ]:
# At this point, Stocks_DailyReturn_Dataframes will contain all the dataframes keyed by their respective sheet names.
Stocks_DailyTradingVolume_Dataframes['0102'].head(2)

,01/01/2001,02/01/2001,03/01/2001,04/01/2001,05/01/2001,08/01/2001,09/01/2001,10/01/2001,11/01/2001,12/01/2001,...,18/12/2001,19/12/2001,20/12/2001,21/12/2001,24/12/2001,25/12/2001,26/12/2001,27/12/2001,28/12/2001,31/12/2001
3I GROUP - TURNOVER BY VALUE,#NA,12074,45256.8,86781,66173.2,49687.9,44567.4,19473.3,79781.7,77786.6,...,28083,8708.7,17294.3,42325.3,668.3,#NA,#NA,2998.9,9700.9,1887
ABB LTD N - TURNOVER BY VALUE,#NA,#NA,131023,141044.5,92481.8,81841.1,207804.8,76380.6,366961.8,160829.8,...,45989.2,50569.9,119078.9,124005.5,#NA,#NA,#NA,52931.6,10858.9,#NA


## Main Variables
The main variables used in the Fama and MacBeth Regressions are **comove, beta, return, size, value and momentum**. These are calculated for all stocks in the index in month t, in the same manner as in Ungeheuer and Weber (2020). This is done for all months. They are stored in *Stocks_Monthly_Dataframes['MMYYYY']*.

### Monthly Frequency of Comovement
We will need to add a column *Comove* to our *Stocks_Monthly_Dataframes['MMYYYY']*. To calculate the frequency of comovement of a particular MMYY, we compare the sign of the weekly return percentage on the stock and on the index for each week of the year prior to that month. Then we calculate in how many percent of the 52 week, the same sign is seen for both return percentages.

#### Weekly Stock Returns

##### Weekly Absolute Return Index of Stocks

First, we will make for each MMYY a dataframe with the absolute return indexes of all the stocks in the past 52 weeks before that month. We will look at the year before the month. We are starting with the first weekday of the previous year (column StartPastYear_Weekday). We will end with the last weekday of the previous year (column EndPastYear_Weekday). We will store them into the *Stocks_WeeklyReturn_Dataframes['MMYYYY]*.

In [ ]:
# Stocks_DailyReturn_Dataframes = {'IndexName': dataframe, ...}
# Initialize the dictionary to store the monthly dataframes
Stocks_WeeklyReturn_Dataframes = {}

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    # Filter the dates DataFrame for the current month and year
    # Assuming 'dates_df' is your DataFrame with 'Month', 'Year', 'Index',
    # 'StartPastYear_Weekday', and 'EndPastYear_Weekday' columns
    matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    # Initialize a list to collect data for the current month
    data_for_month = []

    for _, row in matching_rows.iterrows():
        index_name = row['Index']
        start_date = row['StartPastYear_Weekday']
        end_date = row['EndPastYear_Weekday']

        # Access the corresponding daily returns DataFrame
        daily_returns_df = Stocks_DailyReturn_Dataframes.get(index_name) #Shorter version, MMYY

        if daily_returns_df is not None:

            # Find the columns for start and end dates
            columns = daily_returns_df.columns
            start_col = np.where(columns == pd.Timestamp(start_date))[0][0]
            end_col = np.where(columns == pd.Timestamp(end_date))[0][0]

            # Extract the data between these dates
            data = daily_returns_df.iloc[:, start_col:end_col+1]
            data_for_month.append(data)

    # Concatenate all dataframes for the month (if any) and store in the dictionary
    if data_for_month:
        Stocks_WeeklyReturn_Dataframes[df_name] = pd.concat(data_for_month, axis=1)

In [ ]:
# At this point, Stocks_WeeklyReturn_Dataframes will have all the MMYYYY dataframes as required.
Stocks_WeeklyReturn_Dataframes["012002"].head(2)

,2001-01-01,2001-01-02,2001-01-03,2001-01-04,2001-01-05,2001-01-08,2001-01-09,2001-01-10,2001-01-11,2001-01-12,...,2001-12-18,2001-12-19,2001-12-20,2001-12-21,2001-12-24,2001-12-25,2001-12-26,2001-12-27,2001-12-28,2001-12-31
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,823.58,795.30,758.38,829.48,856.33,844.27,841.79,828.14,852.13,847.95,...,588.90,577.88,574.09,563.69,585.58,585.58,588.48,587.67,594.49,595.39
ABB LTD N - TOT RETURN IND,1957.55,1955.18,1849.66,1925.13,1939.00,1965.14,2007.85,1982.15,1883.67,1853.54,...,771.79,736.28,722.58,771.57,762.66,762.66,761.82,756.93,763.85,764.85


Secondly, we need to calculate the average per week. Note that the year not always starts on a monday and ends on a friday, so we do not always have five daily returns to calculate the weekly average. The 52 weekly averages are stored in *Stocks_WeeklyReturnAverages_Dataframes['MMYYYY']*.

In [ ]:
Stocks_WeeklyReturnAverages_Dataframes = {}

for key, df in Stocks_WeeklyReturn_Dataframes.items():
    Stocks_WeeklyReturnAverages_Dataframes[key] = df.resample('W', axis=1).mean()

In [ ]:
Stocks_WeeklyReturnAverages_Dataframes["012002"].head(2) #toevallig start 2001 op een maandag, maar werkt ook als eerste week niet begint op eerste weekdag

,2001-01-07,2001-01-14,2001-01-21,2001-01-28,2001-02-04,2001-02-11,2001-02-18,2001-02-25,2001-03-04,2001-03-11,...,2001-11-04,2001-11-11,2001-11-18,2001-11-25,2001-12-02,2001-12-09,2001-12-16,2001-12-23,2001-12-30,2002-01-06
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,812.614,842.856,925.586,950.270,943.480,888.874,892.544,884.386,865.820,868.544,...,505.250,541.838,589.898,605.742,591.070,599.432,577.338,578.946,588.360,595.39
ABB LTD N - TOT RETURN IND,1925.304,1938.470,1862.414,1843.808,1825.868,1789.072,1608.592,1524.972,1558.942,1579.038,...,673.882,742.138,806.912,885.270,857.592,867.496,809.318,758.412,761.584,764.85


##### Weekly Relative Return Index of Stocks

For each MMYY, we now have a dataframe with the weekly average stock prices of each stock in the STOXX 600 in the year prior to MMYY. We will compare each week to the previous week in the dataframe. The first week will be compared to the last weekday before the previous year period starts (StartPastYearMinus1_Weekday). We will calculate percent of change in absolute return index. The week-on-week differences are stored in *Stocks_WeeklyReturnPercentages_Dataframes['MMYYYY']*.

In [ ]:
# Placeholder for the new dictionary to store percentage returns DataFrames
Stocks_WeeklyReturnPercentages_Dataframes = {}

for key, weekly_returns_df in Stocks_WeeklyReturnAverages_Dataframes.items():
    # Find the corresponding row in the dates DataFrame
    month, year = int(key[:2]), int(key[2:])
    date_info = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    if not date_info.empty:
        # There should only be one row per month/year, but loop to handle unexpected data
        for _, row in date_info.iterrows():
            index_name = row['Index']
            start_date = row['StartPastYearMinus1_Weekday']

            # Access the daily return on the start date from the corresponding DataFrame
            daily_return_df = Stocks_DailyReturn_Dataframes.get(index_name)
            if daily_return_df is not None:
                start_return = daily_return_df.loc[:, pd.Timestamp(start_date)]

                # Calculate the percentage return for the first week compared to the start date
                first_week_return = weekly_returns_df.iloc[:, 0]

                first_week_percentage = (first_week_return - start_return) / start_return * 100

                # Calculate the week-on-week percentage returns for the rest of the weeks
                weekly_percentage_changes = weekly_returns_df.pct_change(axis=1) * 100

                # Replace the first column with the calculated first week percentage
                weekly_percentage_changes.iloc[:, 0] = first_week_percentage

                # Store the result in the new dictionary
                Stocks_WeeklyReturnPercentages_Dataframes[key] = weekly_percentage_changes

In [ ]:
# Now, Stocks_WeeklyReturnPercentages_Dataframes contains the week-on-week percentage returns for each month/year,
# including the first week's percentage calculated from the StartPastYearMinus1_Weekday.
Stocks_WeeklyReturnPercentages_Dataframes["012002"].head(2)

,2001-01-07,2001-01-14,2001-01-21,2001-01-28,2001-02-04,2001-02-11,2001-02-18,2001-02-25,2001-03-04,2001-03-11,...,2001-11-04,2001-11-11,2001-11-18,2001-11-25,2001-12-02,2001-12-09,2001-12-16,2001-12-23,2001-12-30,2002-01-06
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,-1.331504,3.72157,9.815437,2.666851,-0.714534,-5.787722,0.412882,-0.914017,-2.099310,0.314615,...,-6.933265,7.241564,8.869810,2.685888,-2.422153,1.414722,-3.685823,0.278520,1.626058,1.194847
ABB LTD N - TOT RETURN IND,-1.647263,0.68384,-3.923507,-0.999026,-0.972986,-2.015261,-10.087911,-5.198335,2.227582,1.289079,...,5.157030,10.128776,8.728026,9.710848,-3.126504,1.154862,-6.706429,-6.289987,0.418242,0.428843


#### Weekly Index Returns

##### Weekly Absolute Return Index of EUR600

We follow the same steps as with the stocks. First, we will make for each MMYY a dataframe with the absolute return indexes of the index in the past 52 weeks prior to MMYY. We are starting with the first weekday of the previous year (column StartPastYear_Weekday). We will end with the last weekday of the previous year (column EndPastYear_Weekday). These are stored in *Index_WeeklyReturn_Dataframes['MMYYYY']*.

In [ ]:
# Initialize the dictionary to store the DataFrames
Index_WeeklyReturn_Dataframes = {}

# Create date range
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)
date_range = pd.date_range(start_date, end_date, freq='MS')  # 'MS' is month start frequency

for single_date in date_range:
    month, year = single_date.month, single_date.year
    df_name = f"{month:02d}{year}"

    # Find the corresponding row in the dates DataFrame
    matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    for _, row in matching_rows.iterrows():
        index_name = row['Index']  # Though you mentioned Index, it seems like it's not used directly here.
        start_date = row['StartPastYear_Weekday']
        end_date = row['EndPastYear_Weekday']

        # Filter the Index_DailyReturn_Dataframe to get rows within the start and end date
        mask = (Index_DailyReturn_Dataframe['Date'] >= start_date) & (Index_DailyReturn_Dataframe['Date'] <= end_date)
        filtered_data = Index_DailyReturn_Dataframe.loc[mask, ['Date', 'Return Index ']]

        # Store this filtered data in the Index_WeeklyReturn_Dataframes dictionary
        Index_WeeklyReturn_Dataframes[df_name] = filtered_data

In [ ]:
# At this point, Index_WeeklyReturn_Dataframes will contain your required DataFrames.
Index_WeeklyReturn_Dataframes["012002"].head(2)

,Date,Return Index
1,2001-01-01,100.00
2,2001-01-02,99.09


Now we need to calculate the average per week. Note that the year not always starts on a monday and ends on a friday, so we do not always have five daily returns to calculate the weekly ones. We store these in *Index_WeeklyReturnAverages_Dataframes['MMYYYY'].*

In [ ]:
Index_WeeklyReturnAverages_Dataframes = {}

for key, df in Index_WeeklyReturn_Dataframes.items():
     # Ensure the 'Date' column is of datetime type and set it as the index
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Now that 'Date' is the index, resample the DataFrame to get weekly averages.
    # Note: This assumes that you want to calculate the mean of the 'Return' column on a weekly basis.
    # If your DataFrame contains multiple columns you want to average, you might need to adjust this.
    weekly_avg_df = df.resample('W').mean()

    # Store the resulting DataFrame of weekly averages in the new dictionary
    Index_WeeklyReturnAverages_Dataframes[key] = weekly_avg_df


In [ ]:
Index_WeeklyReturnAverages_Dataframes["012002"].head(2)

,Return Index
Date,
2001-01-07,99.188
2001-01-14,98.274


##### Weekly Relative Return Index of EUR600

For each MMYY, we now have a dataframe with the weekly average return index of the STOXX 600 in the year prior to MMYY. We will compare each week to the previous week in the dataframe. The first week will be compared to the last weekday before the previous year period starts. We will calculate percent of change in total return index. These will be saved in *Index_WeeklyReturnPercentages_Dataframes['MMYY']*.

In [ ]:
Index_WeeklyReturnPercentages_Dataframes = {}

for key, weekly_returns_df in Index_WeeklyReturnAverages_Dataframes.items():
    # Convert key to month and year
    month, year = int(key[:2]), int(key[2:])
    # Find the corresponding information in the dates DataFrame
    date_info = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    if not date_info.empty:
        for _, row in date_info.iterrows():
            index_name = row['Index']
            start_past_year_date = pd.to_datetime(row['StartPastYearMinus1_Weekday'])

            # Access the daily return on the start date from the corresponding DataFrame
            daily_return_df = Index_DailyReturn_Dataframe

            if daily_return_df is not None and start_past_year_date in daily_return_df['Date'].values:
                # Reset index if 'Date' is the index column
                daily_return_df_reset = daily_return_df.reset_index()
                start_return = daily_return_df_reset.loc[daily_return_df_reset['Date'] == start_past_year_date, 'Return Index '].values[0]

                # Reset index for weekly_returns_df to manipulate 'Date' as a column
                weekly_returns_df_reset = weekly_returns_df.reset_index()
                first_week_return = weekly_returns_df_reset.iloc[0]['Return Index ']
                # Calculate the percentage return for the first week compared to the start date
                first_week_percentage = ((first_week_return - start_return) / start_return) * 100

                # Calculate the week-on-week percentage returns for the rest of the weeks
                weekly_returns_df_reset['Weekly_Percentage_Change'] = weekly_returns_df_reset['Return Index '].pct_change() * 100
                # Replace the first percentage change with the calculated first week percentage
                weekly_returns_df_reset.loc[0, 'Weekly_Percentage_Change'] = first_week_percentage

                # Store the result in the new dictionary without resetting index
                Index_WeeklyReturnPercentages_Dataframes[key] = weekly_returns_df_reset.set_index('Date')[['Weekly_Percentage_Change']]

In [ ]:
Index_WeeklyReturnPercentages_Dataframes["012002"].head(2)

,Weekly_Percentage_Change
Date,
2001-01-07,-0.812000
2001-01-14,-0.921482


#### Comparing Weekly Returns

For each stock of EUR600 in MMYY, we now will make a dataframe *Stocks_Monthly_Dataframes['MMYYYY']*. The column *Comove* indicates how many of the weekly returns in Stocks_WeeklyReturnPercentages_Dataframes[MMYYYY] of that particular stock have the same sign as the weekly returns in Index_WeeklyReturnPercentages_Dataframes[MMYYYY].

Let's first make a function to calculate comove based on two series of weekly return percentages:

In [ ]:
# Example function to calculate the Comove value for a stock based on its weekly returns and the index's weekly returns
def calculate_comove(stock_weekly_returns, index_weekly_returns):

    # Calculate the signs (True if positive, False if negative) of the stock's weekly returns
    stock_signs = np.array(stock_weekly_returns > 0)

    # Calculate the signs of the index's weekly returns
    index_signs = np.array(index_weekly_returns > 0)

    # Determine the percentage of weeks where the stock's return sign matches the index's return sign
    matching_signs = np.equal(stock_signs, index_signs)
    matching_signs_percentage = np.mean(matching_signs) * 100
    return matching_signs_percentage

Let's apply the funciton on the data we have so far:

In [ ]:
# Create an empty dictionary to store the dataframes with monthly data of individual stocks
Stocks_Monthly_Dataframes = {}

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)
# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    # Check if we have the data for the MMYYYY in question
    if df_name in Stocks_WeeklyReturnPercentages_Dataframes and df_name in Index_WeeklyReturnPercentages_Dataframes:

        # 1) Get the stock and index dataframes
        stocks_df = Stocks_WeeklyReturnPercentages_Dataframes[df_name]
        index_df = Index_WeeklyReturnPercentages_Dataframes[df_name]

        # 2) Calculate the 'Comove' values
        comove_values = stocks_df.apply(lambda row: calculate_comove(row, index_df.iloc[0]), axis=1) #for each row of the stocks_df (axis=1), apply the function
        # calculate_comove with as first parameter the row itself and as second parameter the first row of the index_df

        # 3) Create a DataFrame for the Comove values with the stock names as the index
        comove_df = pd.DataFrame({
            'Comove': comove_values
        }, index=stocks_df.index)

        # 4) Add to the dictionary
        Stocks_Monthly_Dataframes[df_name] = comove_df


In [ ]:
# This code would result in the Yearly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove
Name,
3I GROUP - TOT RETURN IND,47.169811
ABB LTD N - TOT RETURN IND,60.377358


### Monthly Beta
We will need to add a column *Beta* to our *Stocks_Monthly_Dataframes['MMYYYY']* where we find the beta of each stock in that month. This is calculated using the daily returns in the year prior to MMYY.

#### Daily Stock Returns

##### Daily Absolute Return Index of Stocks

First we find the data we need for each month for stock return index. We do this using the same period as explained in 2.2 about frequency of comovement. In this case, we will use daily return indexes and therefore not calculate weekly averages. Since we do not calculate weekly average, it is easier to already include the day prior to the start date now instead of later. The results are stored in *Stocks_DailyReturnPastYear_Dataframes['MMYYYY']*.

In [ ]:
# Stocks_DailyReturn_Dataframes = {'IndexName': dataframe, ...}
# Initialize the dictionary to store the monthly dataframes
Stocks_DailyReturnPastYear_Dataframes = {}

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    # Filter the dates DataFrame for the current month and year
    # Assuming 'dates_df' is your DataFrame with 'Month', 'Year', 'Index',
    # 'StartPastYear_Weekday', and 'EndPastYear_Weekday' columns
    matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    # Initialize a list to collect data for the current month
    data_for_month = []

    for _, row in matching_rows.iterrows():
        index_name = row['Index']
        start_date = row['StartPastYearMinus1_Weekday'] #HIER WEL ER AL BIJ DOEN MAKKELIJKER
        end_date = row['EndPastYear_Weekday']

        # Access the corresponding daily returns DataFrame
        daily_returns_df = Stocks_DailyReturn_Dataframes.get(index_name)

        if daily_returns_df is not None:

            # Find the columns for start and end dates
            columns = daily_returns_df.columns
            start_col = np.where(columns == pd.Timestamp(start_date))[0][0]
            end_col = np.where(columns == pd.Timestamp(end_date))[0][0]

            # Extract the data between these dates
            data = daily_returns_df.iloc[:, start_col:end_col+1]
            data_for_month.append(data)

    # Concatenate all dataframes for the month (if any) and store in the dictionary
    if data_for_month:
        Stocks_DailyReturnPastYear_Dataframes[df_name] = pd.concat(data_for_month, axis=1)

In [ ]:
Stocks_DailyReturnPastYear_Dataframes["022002"].head(2)

,2001-01-31,2001-02-01,2001-02-02,2001-02-05,2001-02-06,2001-02-07,2001-02-08,2001-02-09,2001-02-12,2001-02-13,...,2002-01-18,2002-01-21,2002-01-22,2002-01-23,2002-01-24,2002-01-25,2002-01-28,2002-01-29,2002-01-30,2002-01-31
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,949.86,936.80,927.67,907.48,914.20,883.26,885.28,854.15,866.79,894.87,...,550.93,553.35,553.84,528.21,528.55,539.08,531.66,525.50,523.93,532.88
ABB LTD N - TOT RETURN IND,1837.97,1802.15,1817.31,1819.32,1829.76,1805.90,1753.66,1736.72,1755.70,1606.52,...,780.79,777.30,762.67,739.33,763.83,799.80,806.49,797.24,741.32,707.26


##### Daily Relative Return Index of Stocks

Then we find the daily return % for stocks, which will be stored in the dataframes *Stocks_DailyReturnPastYearPercentages_Dataframes['MMYYYY']*.

In [ ]:
# Placeholder for the new dictionary to store percentage returns DataFrames
Stocks_DailyReturnPastYearPercentages_Dataframes = {}

for key, weekly_returns_df in Stocks_DailyReturnPastYear_Dataframes.items():
                # Calculate the week-on-week percentage returns for all days -- firt day problem already civered
                weekly_percentage_changes = weekly_returns_df.pct_change(axis=1) * 100

                # Store the result in the new dictionary
                Stocks_DailyReturnPastYearPercentages_Dataframes[key] = weekly_percentage_changes.iloc[:,1:]

In [ ]:
Stocks_DailyReturnPastYearPercentages_Dataframes["022002"].head(2)

,2001-02-01,2001-02-02,2001-02-05,2001-02-06,2001-02-07,2001-02-08,2001-02-09,2001-02-12,2001-02-13,2001-02-14,...,2002-01-18,2002-01-21,2002-01-22,2002-01-23,2002-01-24,2002-01-25,2002-01-28,2002-01-29,2002-01-30,2002-01-31
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,-1.374939,-0.974594,-2.176420,0.740512,-3.384380,0.228698,-3.516402,1.479834,3.239539,-2.171265,...,-3.071834,0.439257,0.088552,-4.627690,0.064368,1.992243,-1.376419,-1.158635,-0.298763,1.708243
ABB LTD N - TOT RETURN IND,-1.948889,0.841217,0.110603,0.573841,-1.303996,-2.892740,-0.965980,1.092865,-8.496896,-4.175485,...,-3.703658,-0.446983,-1.882156,-3.060301,3.313811,4.709163,0.836459,-1.146945,-7.014199,-4.594507


#### Daily Index Returns

##### Daily Absolute Return Index of EUR600

Secondly, we find the data we need for the index analogue to what we did with the stocks. We save the results in *Index_DailyReturnPastYear_Dataframes["MMYYYY"]*. Note that again, since we are using daily returns, we already include the day prior to this period.

In [ ]:
# Initialize the dictionary to store the DataFrames
Index_DailyReturnPastYear_Dataframes = {}

# Create date range
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)
date_range = pd.date_range(start_date, end_date, freq='MS')  # 'MS' is month start frequency

for single_date in date_range:
    month, year = single_date.month, single_date.year
    df_name = f"{month:02d}{year}"

    # Find the corresponding row in the dates DataFrame
    matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    for _, row in matching_rows.iterrows():
        index_name = row['Index']  # Though you mentioned Index, it seems like it's not used directly here.
        start_date = row['StartPastYearMinus1_Weekday'] #HIER ER AL IN VEEL MAKKELIJKER
        end_date = row['EndPastYear_Weekday']

        # Filter the Index_DailyReturn_Dataframe to get rows within the start and end date
        mask = (Index_DailyReturn_Dataframe['Date'] >= start_date) & (Index_DailyReturn_Dataframe['Date'] <= end_date)
        filtered_data = Index_DailyReturn_Dataframe.loc[mask, ['Date', 'Return Index ']]

        # Store this filtered data in the Index_WeeklyReturn_Dataframes dictionary
        Index_DailyReturnPastYear_Dataframes[df_name] = filtered_data

In [ ]:
Index_DailyReturnPastYear_Dataframes["012002"].head(2)

,Date,Return Index
0,2000-12-29,100.0
1,2001-01-01,100.0


##### Daily Relative Return Index of EUR600

We now try to find the day-on-day percentual difference again. We store the results in *Index_DailyReturnPastYearPercentages_Dataframes["MMYYYY"]*.

In [ ]:
Index_DailyReturnPastYearPercentages_Dataframes = {}

for key, weekly_returns_df in Index_DailyReturnPastYear_Dataframes.items():

    # Calculate the week-on-week percentage returns for the rest of the weeks
    weekly_returns_df['Daily_Percentage_Change'] = weekly_returns_df['Return Index '].pct_change() * 100

    # Store the result in the new dictionary without resetting index
    Index_DailyReturnPastYearPercentages_Dataframes[key] = weekly_returns_df.set_index('Date')[['Daily_Percentage_Change']].iloc[1:, :]

In [ ]:
Index_DailyReturnPastYearPercentages_Dataframes["012002"].head(2)

,Daily_Percentage_Change
Date,
2001-01-01,0.00
2001-01-02,-0.91


#### Comparing Daily Returns

Using the daily relative returns of each stock in Stocks_DailyReturnPastYearPercentages_Dataframes['MMYYYY']  with the daily relative returns of the index in Index_DailyReturnPastYearPercentages_Dataframes["MMYYYY"], we calculate beta with its historical regression formula. This formula is set out in the following function:

In [ ]:
# Example function to calculate the beta value for a stock based on its weekly returns and the index's weekly returns
def calculate_beta(stock_daily_returns, market_daily_returns):

    # Calculate covariance between stock and market
    covariance = stock_daily_returns.cov(market_daily_returns)

    # Calculate variance of the market
    variance = market_daily_returns.var()

    # Calculate beta
    beta = covariance / variance

    return beta

When applying the function, results are calculated and stored in the column *Beta* of the *Stocks_Monthly_Dataframes['MMYYYY']*.

In [ ]:
# Loop through the months

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    # Check if we have the data for the year in question
    if df_name in Stocks_DailyReturnPastYearPercentages_Dataframes and df_name in Index_DailyReturnPastYearPercentages_Dataframes:

        # 1) Get the stock and index dataframes
        stocks_df = Stocks_DailyReturnPastYearPercentages_Dataframes[df_name]
        index_df = Index_DailyReturnPastYearPercentages_Dataframes[df_name]

        # 2) Calculate the beta values
        beta_values = stocks_df.apply(lambda row: calculate_beta(row, index_df.iloc[:,0]), axis=1) #for each row of the stocks_df (axis=1), apply the function
        # calculate_beta with as first parameter the row itself and as second parameter the first column of the index_df

        # 3) Add this columnn to the comove values we already had
        Stocks_Monthly_Dataframes[df_name]['Beta'] = beta_values

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1671: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1671: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


In [ ]:
# This code would result in the Yearly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta
Name,,
3I GROUP - TOT RETURN IND,47.169811,1.405906
ABB LTD N - TOT RETURN IND,60.377358,1.675130


### Monthly Returns
We will need to add a column *Return* to our *Stocks_Monthly_Dataframes["MMYYYY.]*.  Instead of the previous year returns we worked with for comove and beta, we now use the returns of the actual month.  

#### Monthly Absolute Return Index of Stocks

Let's first look at the daily returns from the last weekday before the start of the month until last day of the month. We store the results in   *Stocks_DailyReturnThisMonth_Dataframes["MMYYYY"]*. Since we use daily returns, we also already include the return index of the day prior to that month.

In [ ]:
# Stocks_DailyReturn_Dataframes = {'IndexName': dataframe, ...}
# Initialize the dictionary to store the monthly dataframes
Stocks_DailyReturnThisMonth_Dataframes = {}

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    # Filter the dates DataFrame for the current month and year
    # Assuming 'dates_df' is your DataFrame with 'Month', 'Year', 'Index',
    # 'StartPastYear_Weekday', and 'EndPastYear_Weekday' columns
    matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    # Initialize a list to collect data for the current month
    data_for_month = []

    for _, row in matching_rows.iterrows():
        index_name = row['Index']
        start_date = row['StartMonthMinus1_Weekday'] #HIER ER DUS WEL OOK AL BIJ
        end_date = row['EndMonth_Weekday']

        # Access the corresponding daily returns DataFrame
        daily_returns_df = Stocks_DailyReturn_Dataframes.get(index_name)

        if daily_returns_df is not None:

            # Find the columns for start and end dates
            columns = daily_returns_df.columns
            start_col = np.where(columns == pd.Timestamp(start_date))[0][0]
            end_col = np.where(columns == pd.Timestamp(end_date))[0][0]

            # Extract the data between these dates
            data = daily_returns_df.iloc[:, start_col:end_col+1]
            data_for_month.append(data)

    # Concatenate all dataframes for the month (if any) and store in the dictionary
    if data_for_month:
        Stocks_DailyReturnThisMonth_Dataframes[df_name] = pd.concat(data_for_month, axis=1)

In [ ]:
  Stocks_DailyReturnThisMonth_Dataframes["012002"].head(2)

,2001-12-31,2002-01-01,2002-01-02,2002-01-03,2002-01-04,2002-01-07,2002-01-08,2002-01-09,2002-01-10,2002-01-11,...,2002-01-18,2002-01-21,2002-01-22,2002-01-23,2002-01-24,2002-01-25,2002-01-28,2002-01-29,2002-01-30,2002-01-31
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,595.39,595.39,578.56,590.31,605.97,604.36,602.20,591.70,571.40,574.98,...,550.93,553.35,553.84,528.21,528.55,539.08,531.66,525.50,523.93,532.88
ABB LTD N - TOT RETURN IND,764.85,764.85,776.60,836.05,875.67,856.11,845.33,863.46,857.59,862.19,...,780.79,777.30,762.67,739.33,763.83,799.80,806.49,797.24,741.32,707.26


#### Monthly Relative Return Index of Stocks

We take the return of the first and last day, and compare them in order to calculate the monthly stock return. Then we add it to our *Stocks_Monthly_Dataframes["MMYYYY"]* in the column *Return*.

In [ ]:
import pandas as pd
from datetime import datetime

# Assuming Stocks_DailyReturnThisMonth_Dataframes and Stock_Monthly_Dataframes are defined
# Example: Stocks_DailyReturnThisMonth_Dataframes = {'012002': df1, '022002': df2, ...}

# Define the start and end dates and generate monthly periods
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)
monthly_periods = pd.date_range(start_date, end_date, freq='M')

# Iterate over each period
for period in monthly_periods:
    # Format MMYYYY for the dataframe names
    df_name = period.strftime("%m%Y")

    # Find the DataFrame for the current period in Stocks_DailyReturnThisMonth_Dataframes
    if df_name in Stocks_DailyReturnThisMonth_Dataframes:
        daily_return_df = Stocks_DailyReturnThisMonth_Dataframes[df_name]

        # Ensure there is more than one column to calculate return
        if len(daily_return_df.columns) > 1:
            # Get the start and end return
            start_return = daily_return_df.iloc[:, 0]  # First column
            end_return = daily_return_df.iloc[:, -1]  # Last column

            # Calculate the monthly return for each stock
            monthly_return = (end_return - start_return) / start_return * 100

            # Prepare the DataFrame for monthly returns if it does not exist
            if df_name not in Stocks_Monthly_Dataframes:
                Stocks_Monthly_Dataframes[df_name] = pd.DataFrame(index=daily_return_df.index)

            # Add the 'Monthly Return' column to the MMYYYY DataFrame in Stock_Monthly_Dataframes
            Stocks_Monthly_Dataframes[df_name]['Return'] = monthly_return
        else:
            print(f"DataFrame {df_name} does not have enough data to calculate monthly returns.")
    else:
        print(f"DataFrame {df_name} not found in Stocks_DailyReturnThisMonth_Dataframes.")

DataFrame 032024 not found in Stocks_DailyReturnThisMonth_Dataframes.


In [ ]:
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return
Name,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581


### Monthly Size
We will need to add a column *Size* to our *Stocks_Monthly_Dataframes["MMYYYY"]*. The natural logarithm of the average monthly market capitalization in the prior year is taken.

In [ ]:
# Custom function to calculate mean based on conditions
def custom_mean(row):
    try:
        # Convert all entries to numeric, forcing errors to NaN
        numeric_row = pd.to_numeric(row, errors='coerce')
        non_na_values = numeric_row.dropna()
        if non_na_values.empty:
            return pd.NA  # Return NaN if all values are NaN
        if non_na_values.nunique() == 1:
            return non_na_values.iloc[0]  # Return the value if all non-NaN values are the same
        # Otherwise, return the mean of the numeric values
        return non_na_values.mean()
    except Exception as e:
        # Log the exception or handle it as needed
        print(f"Error processing row: {e}")
        return pd.NA

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    if df_name in Stocks_Monthly_Dataframes: #Anders bij bv 3 2024, dit overal toevoegen
        dfMC_name = f"{month:02d}{str(year)[2:]}"  # Use slicing to get the last two digits of the year
        data = Stocks_MonthlyMarketCap_Dataframes[dfMC_name]
        means = data.apply(custom_mean, axis=1)

        # Ensure all element are numeric
        means = means.apply(lambda x: pd.to_numeric(x, errors='coerce'))   # Convert each column to numeric, non-convertible values become NaN
        means = means.apply(lambda x: x if x > 0 else np.nan) # Replace non-positive values with NaN for logging

        # Add the 'Monthly Size' column to the MMYYYY DataFrame in Stock_Monthly_Dataframes
        Stocks_Monthly_Dataframes[df_name]['Size'] =means.apply(np.log).values #nan gets ignored

In [ ]:
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size
Name,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342


### Monthly Value
We will need to add a column *Value* to our *Stocks_Monthly_Dataframes["MMYYYY"]*. The natural logarithm of the average monthly book-to-market ratio in the prior year is taken.

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    if df_name in Stocks_Monthly_Dataframes:

        dfData_name = f"{month:02d}{str(year)[2:]}"  # Use slicing to get the last two digits of the year
        df_marketVal = Stocks_MonthlyMarketVal_Dataframes[dfData_name]
        df_comEq = Stocks_MonthlyComEq_Dataframes[dfData_name]

        # Ensure all data is numeric, non-numeric entries are set to NaN
        df_marketVal = df_marketVal.apply(pd.to_numeric, errors='coerce')
        df_comEq= df_comEq.apply(pd.to_numeric, errors='coerce')

        # Ensure all data is per 1 euro
        df_marketVal *= 1000000

        # Make a common identifier that helps with division
        df_marketVal.reset_index(drop=True, inplace=True)
        df_comEq.reset_index(drop=True, inplace=True)

        # Perform the division, handling divisions by zero and NaN
        result = df_marketVal.div(df_comEq).replace([np.inf, -np.inf], np.nan)

        # Monthly btm ratio gets averaged over year
        means = result.apply(custom_mean, axis=1)

        # Ensure data is ready for logging
        means = means.apply(lambda x: pd.to_numeric(x, errors='coerce'))   # Convert each column to numeric, non-convertible values become NaN
        means = means.apply(lambda x: x if x > 0 else np.nan) # Replace non-positive values with NaN for logging

        # Add the 'Monthly Value' column to the MMYYYY DataFrame in Stock_Monthly_Dataframes
        Stocks_Monthly_Dataframes[df_name]['Value'] = means.apply(np.log).values #nan gets ignored

In [ ]:
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value
Name,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183


### Monthly Momentum
We add a column *Momentum* to our *Stocks_Monthly_Dataframes["MMYYYY"]*. We take the % difference in return index between the last weekday before the prior year starts and the last weekday in the prior year, excluding the last month. So, for example, momentum in Jan 2002 is defined by taking the difference between 31-12-2000 and 30-11-2001, assuming that those are both weekdays.

#### Monthly Absolute Return Index of Stocks

Let's first look at the daily returns from the last weekday before the start of the previous year until last weekday of the previous year. We use the results stored in *Stocks_DailyReturnLastYear_Dataframes["MMYYYY"]*, which we have already made when calculating monthly beta.

In [ ]:
# Show what we already had, since we needed the same variables when calculating beta
Stocks_DailyReturnPastYear_Dataframes["022002"].head(1)

,2001-01-31,2001-02-01,2001-02-02,2001-02-05,2001-02-06,2001-02-07,2001-02-08,2001-02-09,2001-02-12,2001-02-13,...,2002-01-18,2002-01-21,2002-01-22,2002-01-23,2002-01-24,2002-01-25,2002-01-28,2002-01-29,2002-01-30,2002-01-31
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,949.86,936.8,927.67,907.48,914.2,883.26,885.28,854.15,866.79,894.87,...,550.93,553.35,553.84,528.21,528.55,539.08,531.66,525.5,523.93,532.88


#### Monthly Relative Return Index of Stocks (Momentum)

In order to calculate momentum, we need two different dates. Firstly, we need the last weekday before the prior year starts, which will always be defined as the first row in *Stocks_DailyReturnLastYear_Dataframes["MMYYYY"]*. Secondly, we need the last weekday of the prior year, but we want to exclude the last month. So we want the last weekday of two months prior. This is defined as *EndPastYearExclPastMonth_Weekday* column in the *dates* dataframe with MMYY index. We take the both dates that we need out of the dataframe and calculate the percentual difference.

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    year = period.year
    month = period.month
    matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]    # Filter the dates DataFrame for the current month and year

    df_name = f"{month:02d}{year}" # Format the DataFrame name as MM20YY
    data_for_month = []

    for _, row in matching_rows.iterrows():
        end_date_row = row['EndPastYearExclPastMonth_Weekday'] # This is the date

        daily_returns_df = Stocks_DailyReturnPastYear_Dataframes.get(df_name)

        if daily_returns_df is not None:
            columns = daily_returns_df.columns
            start_return = daily_returns_df.iloc[:, 0]

            if pd.Timestamp(end_date_row) in columns:
                end_col = np.where(columns == pd.Timestamp(end_date_row))[0][0]
                end_return = daily_returns_df.iloc[:, end_col]
                monthly_momentum = (end_return - start_return) / start_return * 100

                if df_name not in Stocks_Monthly_Dataframes:
                    Stocks_Monthly_Dataframes[df_name] = pd.DataFrame(index=daily_returns_df.index)

                Stocks_Monthly_Dataframes[df_name]['Momentum'] = monthly_momentum
            else:
                print(f"End date {end_date_row} not found in columns.") #Could not find this date in Stocks_DailyReturnPastYear_Dataframes
        else:
            print(f"DataFrame for {df_name} not available.") #Could not find this in Stocks_DailyReturnPastYear_Dataframes


In [ ]:
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum
Name,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242


## Other Measurements of Dependency and Volatility
These are measurements of downside and upside risk (**upside and downside beta, UTD and LTD**), idiosyncratic risk (**idiosyncratic volatility, minimum return and maximum return**), illiquidy (**Amihud illiquidity ratio**) and trading activity (**turnover and difference in turnover**). These are calculated for all stocks in the index in month t. This is done for all months. They are stored in *Stocks_Monthly_Dataframes['MMYYYY']*.

In [ ]:
# TODO Extra: Plot all measurements van dependency and volatiliy over time, average genomen van alle aandelen in die maand. Maken zelfde beweging?
# Plot mss beter met coefficienten cross-regression.

### Measurements of Downside and Upside Risk

#### Monthly Downside Beta
We add a column *Beta_Downside* to our *Stocks_Monthly_Dataframes['MMYYYY']*. We use the daily returns of stock and index in the year prior to the month t. We calculate the correlation of returns, but only returns in days where the stock return is below the mean index return, as in Chen et al. (1986)

We do so by using the daily relative returns of each stock in Stocks_DailyReturnPastYearPercentages_Dataframes['MMYYYY'] with the daily relative returns of the index in Index_DailyReturnPastYearPercentages_Dataframes["MMYYYY"], which were made when calculating beta.

In [ ]:
Stocks_DailyReturnPastYearPercentages_Dataframes['012002'].head(1)

,2001-01-01,2001-01-02,2001-01-03,2001-01-04,2001-01-05,2001-01-08,2001-01-09,2001-01-10,2001-01-11,2001-01-12,...,2001-12-18,2001-12-19,2001-12-20,2001-12-21,2001-12-24,2001-12-25,2001-12-26,2001-12-27,2001-12-28,2001-12-31
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,0.0,-3.433789,-4.642273,9.375247,3.236968,-1.408336,-0.293745,-1.621545,2.896853,-0.490535,...,-0.215192,-1.871285,-0.655846,-1.811563,3.88334,0.0,0.495235,-0.137643,1.160515,0.15139


In [ ]:
Index_DailyReturnPastYearPercentages_Dataframes['012002'].head(1)

,Daily_Percentage_Change
Date,
2001-01-01,0.0


We use the following function:

In [ ]:
# Example function to calculate the Downside Beta value for a stock based on its weekly returns and the index's daily returns
def calculate_downside_beta(stock_daily_returns, market_daily_returns):
    """
    Calculate the downside beta for stock returns compared to market returns.

    Parameters:
    - stock_daily_returns (pd.Series): A pandas Series of daily returns for the stock.
    - market_daily_returns (pd.Series): A pandas Series of daily returns for the market.

    Returns:
    - float: The calculated downside beta of the stock.
    """
    # Ensure both input Series have the same index for proper alignment
    if not stock_daily_returns.index.equals(market_daily_returns.index):
        raise ValueError("Indices of stock and market returns do not match.")

    # Calculate the mean market return
    mean_market_return = market_daily_returns.mean()

    # Filter days where the stock return is below the mean market return
    filtered_indices = stock_daily_returns < mean_market_return
    filtered_stock_returns = stock_daily_returns[filtered_indices]
    filtered_market_returns = market_daily_returns[filtered_indices]

    downside_beta = calculate_beta(filtered_stock_returns, filtered_market_returns)

    return downside_beta

When applying the function, results are calculated and stored in the column *Beta_Downside* of the *Stocks_Monthly_Dataframes['MMYYYY']*.

In [ ]:
# Loop through the months

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    # Check if we have the data for the year in question
    if df_name in Stocks_DailyReturnPastYearPercentages_Dataframes and df_name in Index_DailyReturnPastYearPercentages_Dataframes:

        # 1) Get the stock and index dataframes
        stocks_df = Stocks_DailyReturnPastYearPercentages_Dataframes[df_name]
        index_df = Index_DailyReturnPastYearPercentages_Dataframes[df_name]

        # 2) Calculate the beta values
        beta_values = stocks_df.apply(lambda row: calculate_downside_beta(row, index_df.iloc[:,0]), axis=1) #for each row of the stocks_df (axis=1), apply the function
        # calculate_beta with as first parameter the row itself and as second parameter the first column of the index_df

        # 3) Add this columnn to the comove values we already had
        Stocks_Monthly_Dataframes[df_name]['Beta_Downside'] = beta_values

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1671: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1671: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1671: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside
Name,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205


#### Monthly Upside Beta
We add a column *Beta_Upside* to our *Stocks_Monthly_Dataframes['MMYYYY']*. We use the daily returns of stock and index in the year prior to the month t. We calculate the correlation of returns, but only returns in days where the stock return is above the mean index return, as in Chen et al. (1986)

We work analogous to Downside Beta.

In [ ]:
# Example function to calculate the Downside Beta value for a stock based on its daily returns and the index's daily returns
def calculate_upside_beta(stock_daily_returns, market_daily_returns):
    """
    Calculate the upside beta for stock returns compared to market returns.

    Parameters:
    - stock_daily_returns (pd.Series): A pandas Series of daily returns for the stock.
    - market_daily_returns (pd.Series): A pandas Series of daily returns for the market.

    Returns:
    - float: The calculated upside beta of the stock.
    """
    # Ensure both input Series have the same index for proper alignment
    if not stock_daily_returns.index.equals(market_daily_returns.index):
        raise ValueError("Indices of stock and market returns do not match.")

    # Calculate the mean market return
    mean_market_return = market_daily_returns.mean()

    # Filter days where the stock return is above the mean market return
    filtered_indices = stock_daily_returns > mean_market_return
    filtered_stock_returns = stock_daily_returns[filtered_indices]
    filtered_market_returns = market_daily_returns[filtered_indices]

    upside_beta = calculate_beta(filtered_stock_returns, filtered_market_returns)

    return upside_beta

In [ ]:
# Loop through the months

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    # Check if we have the data for the year in question
    if df_name in Stocks_DailyReturnPastYearPercentages_Dataframes and df_name in Index_DailyReturnPastYearPercentages_Dataframes:

        # 1) Get the stock and index dataframes
        stocks_df = Stocks_DailyReturnPastYearPercentages_Dataframes[df_name]
        index_df = Index_DailyReturnPastYearPercentages_Dataframes[df_name]

        # 2) Calculate the beta values
        beta_values = stocks_df.apply(lambda row: calculate_upside_beta(row, index_df.iloc[:,0]), axis=1) #for each row of the stocks_df (axis=1), apply the function
        # calculate_beta with as first parameter the row itself and as second parameter the first column of the index_df

        # 3) Add this columnn to the comove values we already had
        Stocks_Monthly_Dataframes[df_name]['Beta_Upside'] = beta_values

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1671: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1671: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside
Name,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786


#### Monthly Lower Tail Dependence
We add a column *LTD* to our *Stocks_Monthly_Dataframes['MMYYYY']*. We estimate the Kendall tau's correlation coefficient of the stock and market daily returns in the year prior to the month t. Then we estimate the theta parameter of the Clayton copula function derived from Kendall's tau, and calculate its lower tail dependence coefficient. This is based on Chabi-Yo et al. (2017), who use multiple copula functions to calculate LTD.

In [ ]:
# Example function to calculate the LTD value for a stock based on its daily returns and the index's daily returns
def calculate_LTD(stock_daily_returns, market_daily_returns):
    """
    Calculate the Lower Tail Dependence for stock returns compared to market returns.

    Parameters:
    - stock_daily_returns (pd.Series): A pandas Series of daily returns for the stock.
    - market_daily_returns (pd.Series): A pandas Series of daily returns for the market.

    Returns:
    - float: The calculated LTD.
    """
    # Ensure both input Series have the same index for proper alignment
    if not stock_daily_returns.index.equals(market_daily_returns.index):
        raise ValueError("Indices of stock and market returns do not match.")

    # Calculate Kendall's tau rank correlation coefficient, so we don't rely on marginal distributions
    kendalls_tau = stock_daily_returns.corr(market_daily_returns, method='kendall')

    # Calculate the Clayton copula parameter theta from Tau, so we don't have to actually fit the data
    if kendalls_tau == 1:
        theta = float('inf')  # To handle perfect correlation
    else:
        theta = 2 * kendalls_tau / (1 - kendalls_tau)

    # Calculate the lower tail dependence from the Clayton copula parameter, with the formula
    ltd = 2 ** (-1 / theta) if theta > 0 else 0  # Theta must be positive for LTD to be meaningful

    return ltd

In [ ]:
# Loop through the months

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    # Check if we have the data for the year in question
    if df_name in Stocks_DailyReturnPastYearPercentages_Dataframes and df_name in Index_DailyReturnPastYearPercentages_Dataframes:

        # 1) Get the stock and index dataframes
        stocks_df = Stocks_DailyReturnPastYearPercentages_Dataframes[df_name]
        index_df = Index_DailyReturnPastYearPercentages_Dataframes[df_name]

        # 2) Calculate the LTD
        ltd_values = stocks_df.apply(lambda row: calculate_LTD(row, index_df.iloc[:,0]), axis=1) #for each row of the stocks_df (axis=1), apply the function
        # with as first parameter the row itself and as second parameter the first column of the index_df

        # 3) Add this columnn to the comove values we already had
        Stocks_Monthly_Dataframes[df_name]['LTD'] = ltd_values

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD
Name,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230


#### Monthly Upper Tail Dependence
We add a column *UTD* to our *Stocks_Monthly_Dataframes['MMYYYY']*. We estimate the Kendall tau's correlation coefficient of the stock and market daily returns in the year prior to the month t. Then we estimate the alpha parameter of the Gumbel copula function derived from Kendall's tau, and calculate its upper tail dependence coefficient. This is based on Chabi-Yo et al. (2017), who use multiple copula functions to calculate UTD.

In [ ]:
# Example function to calculate the UTD value for a stock based on its daily returns and the index's daily returns
def calculate_UTD(stock_daily_returns, market_daily_returns):
    """
    Calculate the Upper Tail Dependence for stock returns compared to market returns.

    Parameters:
    - stock_daily_returns (pd.Series): A pandas Series of daily returns for the stock.
    - market_daily_returns (pd.Series): A pandas Series of daily returns for the market.

    Returns:
    - float: The calculated UTD.
    """
    # Ensure both input Series have the same index for proper alignment
    if not stock_daily_returns.index.equals(market_daily_returns.index):
        raise ValueError("Indices of stock and market returns do not match.")

     # Calculate Kendall's tau rank correlation coefficient
    kendalls_tau = stock_daily_returns.corr(market_daily_returns, method='kendall')

    # Calculate the Gumbel copula parameter alpha from Kendall's tau
    alpha = 1 / (1 - kendalls_tau) if kendalls_tau != 1 else float('inf')  # Handle the edge case of perfect correlation

    # Calculate the upper tail dependence from the Gumbel copula parameter
    utd = 2 - 2 ** (1 / alpha) if alpha != float('inf') else 1  # For perfect correlation, UTD is 1

    return utd

In [ ]:
# Loop through the months

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    # Check if we have the data for the year in question
    if df_name in Stocks_DailyReturnPastYearPercentages_Dataframes and df_name in Index_DailyReturnPastYearPercentages_Dataframes:

        # 1) Get the stock and index dataframes
        stocks_df = Stocks_DailyReturnPastYearPercentages_Dataframes[df_name]
        index_df = Index_DailyReturnPastYearPercentages_Dataframes[df_name]

        # 2) Calculate the UTD
        utd_values = stocks_df.apply(lambda row: calculate_UTD(row, index_df.iloc[:,0]), axis=1) #for each row of the stocks_df (axis=1), apply the function
        # with as first parameter the row itself and as second parameter the first column of the index_df

        # 3) Add this columnn to the comove values we already had
        Stocks_Monthly_Dataframes[df_name]['UTD'] = utd_values

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD
Name,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152


### Measurements of Idiosyncratic Risk

#### Monthly Idiosyncratic Volatility
We add a column *IdioVol* to our *Stocks_Monthly_Dataframes['MMYYYY']*. Using the daily returns of in the prior month, we calculate the standard deviation of residuals in the regression Rstock, t = α + β * Rmarket, t + ϵt, as in Fama and French (1992).

In [ ]:
Index_DailyReturnPastYearPercentages_Dataframes['012002'].head(2)

,Daily_Percentage_Change
Date,
2001-01-01,0.00
2001-01-02,-0.91


We first need the return percentages of the past month instead of the past year, since we will need them later as well. Let's see if we can select the correct part of *Stocks_DailyReturnPastYearPercentages_Dataframes['MMYYYY']* and *Index_DailyReturnPastYearPercentages_Dataframes["MMYYYY"]*. For each month, we will look up the start date of the needed period in the *dates* dataframe that is on the row with Year and Month column having the correct int values, and the column StartPastMonth_Weekday.  For each month, we will look up the end date of the needed period in the *dates* dataframe that is on the row with Year and Month column having the correct int values, and the column EndPastMonth_Weekday. Then we will use the start and end date to look up the correct columns in the *Stocks_DailyReturnPastYearPercentages_Dataframes['MMYYYY']* and *Index_DailyReturnPastYearPercentages_Dataframes["MMYYYY"]*. Both columns, and all columns in between are needed. We make a copy of the returns in the past month in the dictionairies *Stocks_DailyReturnPastMonthPercentages_Dataframes['MMYYYY']* and *Index_DailyReturnPastMonthPercentages_Dataframes["MMYYYY"]*.

In [ ]:
# Initialize dictionaries for monthly returns
Stocks_DailyReturnPastMonthPercentages_Dataframes = {}
Index_DailyReturnPastMonthPercentages_Dataframes = {}

# Loop through the months
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)
# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    year = period.year
    month = period.month

     # Access the dates dataframe to get start and end dates
    start_date_series = dates.loc[(dates['Year'] == year) & (dates['Month'] == month), 'StartPastMonth_Weekday']
    end_date_series = dates.loc[(dates['Year'] == year) & (dates['Month'] == month), 'EndPastMonth_Weekday']

    # Check if dates are present in the series
    if not start_date_series.empty and not end_date_series.empty:
        # Convert to timestamp
        start_date = pd.Timestamp(start_date_series.iloc[0])
        end_date = pd.Timestamp(end_date_series.iloc[0])

        # Format the key to match the dictionary keys
        key = f"{month:02d}{year}"

        #For stocks
        # Get columns from the dataframe
        columns = Stocks_DailyReturnPastYearPercentages_Dataframes[key].columns
        # Find indices using np.where
        start_idx = np.where(columns == start_date)[0]
        end_idx = np.where(columns == end_date)[0]
        # Check if indices are found
        if start_idx.size > 0 and end_idx.size > 0:
            start_col = start_idx[0]
            end_col = end_idx[0]
            # Extract the relevant slice from the year dataframe
            stocks_monthly_returns = Stocks_DailyReturnPastYearPercentages_Dataframes[key].iloc[:, start_col:end_col+1]
            # Store in the new dictionaries
            Stocks_DailyReturnPastMonthPercentages_Dataframes[key] = stocks_monthly_returns.copy()
        else:
            print(f"Dates not found in columns for {key}: Start Date - {start_date}, End Date - {end_date}")

        #For index
        # Extract the relevant rows between the start and end dates
        index_monthly_returns = Index_DailyReturnPastYearPercentages_Dataframes[key].loc[start_date:end_date, 'Daily_Percentage_Change']
        Index_DailyReturnPastMonthPercentages_Dataframes[key] = index_monthly_returns.copy()
    else:
        print(f"Missing date information for {year}-{month}")

Missing date information for 2024-3


In [ ]:
Stocks_DailyReturnPastMonthPercentages_Dataframes['012002'].head(2)

,2001-12-03,2001-12-04,2001-12-05,2001-12-06,2001-12-07,2001-12-10,2001-12-11,2001-12-12,2001-12-13,2001-12-14,...,2001-12-18,2001-12-19,2001-12-20,2001-12-21,2001-12-24,2001-12-25,2001-12-26,2001-12-27,2001-12-28,2001-12-31
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,-2.464233,1.160585,5.283979,1.868976,-2.904385,-1.073617,-3.322592,-1.982626,-0.116289,1.305368,...,-0.215192,-1.871285,-0.655846,-1.811563,3.883340,0.0,0.495235,-0.137643,1.160515,0.151390
ABB LTD N - TOT RETURN IND,-3.413806,5.737540,5.293167,2.902753,-1.225702,-8.358954,-2.375127,1.303922,-1.359395,-1.551947,...,-2.285273,-4.600992,-1.860705,6.779872,-1.154788,0.0,-0.110141,-0.641884,0.914219,0.130916


In [ ]:
Index_DailyReturnPastMonthPercentages_Dataframes['012002'].head(2)

Date
2001-12-03   -0.376214
2001-12-04    1.230357
Name: Daily_Percentage_Change, dtype: float64

Now we can calculate the idiosyncratic volatility.

In [ ]:
# Example function to calculate the Idiosyncrastic Volatilty value for a stock based on its daily returns and the index's daily returns
def calculate_IdioVol(stock_daily_returns, market_daily_returns):
    """
    Calculate the Idiosyncratic Volatility for stock returns compared to market returns.
    Using the daily returns of in the prior month, we calculate the standard deviation of residuals in the regression Rstock, t = α + β * Rmarket, t + ϵt.

    Parameters:
    - stock_daily_returns (pd.Series): A pandas Series of daily returns for the stock.
    - market_daily_returns (pd.Series): A pandas Series of daily returns for the market.

    Returns:
    - float: The calculated idioVol.
    """
    # Ensure both input Series have the same index for proper alignment
    if not stock_daily_returns.index.equals(market_daily_returns.index):
        raise ValueError("Indices of stock and market returns do not match.")

    # Add a constant to the market returns to account for the intercept (alpha)
    market_daily_returns = sm.add_constant(market_daily_returns)

    # Perform OLS regression
    model = sm.OLS(stock_daily_returns, market_daily_returns)
    results = model.fit()

    # Calculate residuals
    residuals = results.resid

    # Calculate the standard deviation of the residuals (Idiosyncratic Volatility)
    idioVol = np.std(residuals, ddof=1)  # delta degrees of freedom=1 for sample standard deviation (Bessel Correction)

    return idioVol

In [ ]:
# Loop through the months

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    # Check if we have the data for the year in question
    if df_name in Stocks_DailyReturnPastMonthPercentages_Dataframes and df_name in Index_DailyReturnPastMonthPercentages_Dataframes:

        # 1) Get the stock and index dataframes
        stocks_df = Stocks_DailyReturnPastMonthPercentages_Dataframes[df_name]
        index_df = Index_DailyReturnPastMonthPercentages_Dataframes[df_name]

        # 2) Calculate the Idio Vol
        idioVol_values = stocks_df.apply(lambda row: calculate_IdioVol(row, index_df), axis=1)


        # 3) Add this columnn to the comove values we already had
        Stocks_Monthly_Dataframes[df_name]['IdioVol'] = idioVol_values

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,IdioVol
Name,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,1.846227
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,2.576129


#### Monthly Minimum Return
We add a column *Min* to our *Stocks_Monthly_Dataframes['MMYYYY']*. Using the daily returns of in the prior month, we take the minimum as in Bali et. al (2011).

In [ ]:
# Loop through the months

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    # Check if we have the data for the year in question
    if df_name in Stocks_DailyReturnPastMonthPercentages_Dataframes and df_name in Index_DailyReturnPastMonthPercentages_Dataframes:

        # 1) Get the stock and index dataframes
        stocks_df = Stocks_DailyReturnPastMonthPercentages_Dataframes[df_name]

        # 2) Calculate the minimum values
        min_values = stocks_df.apply(lambda row: row.min(), axis=1)


        # 3) Add this columnn to the comove values we already had
        Stocks_Monthly_Dataframes[df_name]['Min'] = min_values

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,IdioVol,Min
Name,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,1.846227,-3.322592
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,2.576129,-8.358954


In [ ]:
# forgot to multiply by -1, fixed this in models notebook

#### Monthly Maximum Return
We add a column *Max* to our *Stocks_Monthly_Dataframes['MMYYYY']*. Using the daily returns of in the prior month, we take the maximum as in Bali et. al (2011).

In [ ]:
# Loop through the months

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    # Check if we have the data for the year in question
    if df_name in Stocks_DailyReturnPastMonthPercentages_Dataframes and df_name in Index_DailyReturnPastMonthPercentages_Dataframes:

        # 1) Get the stock and index dataframes
        stocks_df = Stocks_DailyReturnPastMonthPercentages_Dataframes[df_name]

        # 2) Calculate the maximum values
        max_values = stocks_df.apply(lambda row: row.max(), axis=1)


        # 3) Add this columnn to the comove values we already had
        Stocks_Monthly_Dataframes[df_name]['Max'] = max_values

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,IdioVol,Min,Max
Name,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,1.846227,-3.322592,5.283979
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,2.576129,-8.358954,6.779872


### Measurements of Illiquidity

#### Monthly Amihud Illiquidity Ratio
We add a column *Illiqudity* to our *Stocks_Monthly_Dataframes["MMYYYY"]*. In each day of the prior year, the daily Amihud (2002) illiquidity ratio is calculated by dividng the absolute value of the daily return with the trading volume. The average of the daily Amihud illiquidity ratios in the prior year is taken.

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)  # DEBUG end_date = datetime(2002, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYYYY
    df_name = f"{month:02d}{year}"

    if df_name in Stocks_Monthly_Dataframes: #Anders bij bv 3 2024, dit overal toevoegen

        # Get returns, ensure there are only Nan and numeric values,
        # Divide numeric values by 100 for decimal form and take absolute value, while Nan stays Nan
        returns_df = Stocks_DailyReturnPastYearPercentages_Dataframes[df_name] #Calculated by beta
        returns_df = returns_df.apply(pd.to_numeric, errors='coerce').abs() / 100

        # Get trading activity and ensure there are only Nan and numeric values
        short_df_name = df_name[:2] + df_name[4:] #In MMYY format instead of MMYYYY
        turnover_df = Stocks_DailyTradingVolume_Dataframes[short_df_name]
        turnover_df = turnover_df.apply(pd.to_numeric, errors='coerce')

        # Calculate the Daily Illiquidity Ratio by dividing the two dataframes element-wise
        # Note that they do not have same row index, but are in the same order.
        illiquidity_ratio = returns_df.div(turnover_df.values, axis=0)

        # Take yearly average over daily Iliquidty Ratio (one per row)
        monthly_illiquidity = illiquidity_ratio.mean(axis=1)

         # Add the 'Monthly Illiquidty' column to the MMYYYY DataFrame in Stock_Monthly_Dataframes
        Stocks_Monthly_Dataframes[df_name]['Illiquidity'] = monthly_illiquidity.fillna(np.nan).values

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,IdioVol,Min,Max,Illiquidity
Name,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,1.846227,-3.322592,5.283979,9.528294e-07
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,2.576129,-8.358954,6.779872,3.783419e-07


These small numbers indicate that the price impact per unit of volume is minimal, which is typical for highly liquid stocks with high trading volumes. The Amihud ratio is designed to reflect the price impact of trading; hence, a smaller ratio signifies lower price impact (i.e., higher liquidity).

### Measurements of Trading Activity

#### Monthly Turnover
We add a column *Ln(Turnover)* to our *Stocks_Monthly_Dataframes["MMYYYY"]*. The natural logarithm of the monthly turn-over rate of a stock in the prior month is taken. This is based on the work of Gervais et al. (2001).

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYYYY
    df_name = f"{month:02d}{year}"
    dfTurnover_name = f"{month:02d}{str(year)[2:]}"  # Use slicing to get the last two digits of the year

    if df_name in Stocks_Monthly_Dataframes and dfTurnover_name in Stocks_MonthlyTurnover_Dataframes:
            #Get correct dataframe
            turnover = Stocks_MonthlyTurnover_Dataframes[dfTurnover_name]

            # Ensure turnover is a DataFrame and convert all elements to numeric
            if isinstance(turnover, pd.DataFrame):
                turnover = turnover.apply(lambda x: pd.to_numeric(x, errors='coerce'))   # Convert each column to numeric, non-convertible values become NaN
                turnover = turnover.applymap(lambda x: x if x > 0 else np.nan) # Replace non-positive values with NaN for logging

                # Apply natural logarithm to each column, np.log safely ignores NaN values
                ln_turnover = turnover.apply(np.log)

                Stocks_Monthly_Dataframes[df_name]['Ln(Turnover)'] = ln_turnover.iloc[:, -1].values  # Take the last column

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,IdioVol,Min,Max,Illiquidity,Ln(Turnover)
Name,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,1.846227,-3.322592,5.283979,9.528294e-07,18.533483
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,2.576129,-8.358954,6.779872,3.783419e-07,18.920277


#### Monthly Difference in Turnover
We add a column *Diff in Ln(Turnover)* to our *Stocks_Monthly_Dataframes["MMYYYY"]*. The natural logarithm of the monthly turn-over rate of a stock in two prior months is taken, and the difference between the month prior and two months prior is calculated. This is based on the work of Gervais et al. (2001).

In [ ]:
# TODO is this percentual or aboslute difference?

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYYYY
    df_name = f"{month:02d}{year}"
    dfTurnover_name = f"{month:02d}{str(year)[2:]}"  # Use slicing to get the last two digits of the year

    if df_name in Stocks_Monthly_Dataframes and dfTurnover_name in Stocks_MonthlyTurnover_Dataframes:
            #Get correct dataframe
            turnover = Stocks_MonthlyTurnover_Dataframes[dfTurnover_name]

            # Ensure turnover is a DataFrame and convert all elements to numeric
            if isinstance(turnover, pd.DataFrame):
                turnover = turnover.apply(lambda x: pd.to_numeric(x, errors='coerce'))   # Convert each column to numeric, non-convertible values become NaN
                turnover = turnover.applymap(lambda x: x if x > 0 else np.nan) # Replace non-positive values with NaN for logging

                # Apply natural logarithm to each column, np.log safely ignores NaN values
                ln_turnover = turnover.apply(np.log)

                # Take the difference
                difference = ln_turnover.iloc[:, -1] - ln_turnover.iloc[:, -2]
                difference = difference.values

                Stocks_Monthly_Dataframes[df_name]['Diff in Ln(Turnover)'] = difference

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,IdioVol,Min,Max,Illiquidity,Ln(Turnover),Diff in Ln(Turnover)
Name,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,1.846227,-3.322592,5.283979,9.528294e-07,18.533483,-0.185606
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,2.576129,-8.358954,6.779872,3.783419e-07,18.920277,-0.386665


## Other Benchmarks, Fixed Effects and Skipped Month
These are other benchmarks (**short-, medium- and long-term momentum, operating profitability and investment**), as well as fixed effects (**industry, exchange and size dummies**) and an alternative comove measure with skipped month (**Comove_skippedMonth**). These are calculated for all stocks in the index in month t. This is done for all months. They are stored in *Stocks_Monthly_Dataframes['MMYYYY']*.

### Other Benchmarks

#### Monthly Short-Term Momentum
We add a column *Momentum_ST* to our *Stocks_Monthly_Dataframes['MMYYYY']*. We use the daily returns of the stock in the last weekday before the prior month starts and the last weekday of the prior month to calculate the percentual difference.

In [ ]:
# Loop through the months
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)
# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    year = period.year
    month = period.month

     # Access the dates dataframe to get start and end dates
    start_date_series = dates.loc[(dates['Year'] == year) & (dates['Month'] == month), 'StartPastMonthMinus1_Weekday']
    end_date_series = dates.loc[(dates['Year'] == year) & (dates['Month'] == month), 'EndPastMonth_Weekday']

    # Check if dates are present in the series
    if not start_date_series.empty and not end_date_series.empty:
        # Convert to timestamp
        start_date = pd.Timestamp(start_date_series.iloc[0])
        end_date = pd.Timestamp(end_date_series.iloc[0])

        # Format the key to match the dictionary keys
        key = f"{month:02d}{year}" # YYYYY

        # Get columns from the dataframe where the absolute return indexes are.
        columns = Stocks_DailyReturnPastYear_Dataframes[key].columns #We dont use the normal DailyReturn of import since these are per quarter
        # Find indices using np.where
        start_idx = np.where(columns == start_date)[0]
        end_idx = np.where(columns == end_date)[0]
        # Check if indices are found
        if start_idx.size > 0 and end_idx.size > 0:
            # Take first columns with that index. Extract values.
            start_col = start_idx[0]
            end_col = end_idx[0]
            # Extract the relevant slice from the year dataframe
            start_values = Stocks_DailyReturnPastYear_Dataframes[key].iloc[:, start_col]
            end_values =  Stocks_DailyReturnPastYear_Dataframes[key].iloc[:, end_col]
            # Store momentum
            Stocks_Monthly_Dataframes[key]['Momentum_ST'] = ((end_values - start_values) / start_values) * 100

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,IdioVol,Min,Max,Illiquidity,Ln(Turnover),Diff in Ln(Turnover),Momentum_ST
Name,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,1.846227,-3.322592,5.283979,9.528294e-07,18.533483,-0.185606,1.045432
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,2.576129,-8.358954,6.779872,3.783419e-07,18.920277,-0.386665,-7.212180


#### Monthly Medium-Term Momentum
We add a column *Momentum_MT* to our *Stocks_Monthly_Dataframes['MMYYYY']*. This is the same as the already calculated momentum.

In [ ]:
# Loop through the months

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    if df_name in Stocks_Monthly_Dataframes:
        Stocks_Monthly_Dataframes[df_name]['Momentum_MT'] = Stocks_Monthly_Dataframes[df_name]['Momentum']

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,IdioVol,Min,Max,Illiquidity,Ln(Turnover),Diff in Ln(Turnover),Momentum_ST,Momentum_MT
Name,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,1.846227,-3.322592,5.283979,9.528294e-07,18.533483,-0.185606,1.045432,-28.455038
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,2.576129,-8.358954,6.779872,3.783419e-07,18.920277,-0.386665,-7.212180,-57.891242


#### Monthly Long-Term Momentum
We add a column *Momentum_LT* to our *Stocks_Monthly_Dataframes['MMYYYY']*. We use the daily returns of stock and index in the two years before the year prior.

##### Monthly Absolute Return Index of Stocks

Firstly, the *Stocks_MonthlyReturn_Dataframes[MMYY]* are still ordered per quarter instead of per month. Let's fix that, by making *Stocks_MonthlyReturnPastThreeYears_Dataframes[MMYYYY]* for each month. This will include all the absolute return indexes that are in the data. As discussed below, we will not need all of these for this variable, but we store them for efficiency purposes since we do need all of the data for other variables.

In [ ]:
# Initialize the dictionary to store the monthly dataframes
Stocks_MonthlyReturnPastThreeYears_Dataframes = {}

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    # Filter the dates DataFrame for the current month and year
    # Assuming 'dates_df' is your DataFrame with 'Month', 'Year', 'Index',
    # 'StartPastYear_Weekday', and 'EndPastYear_Weekday' columns
    matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    # Initialize a list to collect data for the current month
    data_for_month = []

    for _, row in matching_rows.iterrows():
        index_name = row['Index'] #This is the sheet name
         # This date minus two years
        start_date = pd.to_datetime(row['StartPastYear']) - pd.DateOffset(years=2)
        # This date but set to the 1st of this month
        end_date = pd.to_datetime(row['EndPastYear']).replace(day=1)

        # Access the corresponding daily returns DataFrame
        daily_returns_df = Stocks_MonthlyReturn_Dataframes.get(index_name) #Shorter version, MMYY

        if daily_returns_df is not None:
            # Find the columns for start and end dates
            columns = daily_returns_df.columns
            start_col = np.where(columns == pd.Timestamp(start_date))[0][0]
            end_col = np.where(columns == pd.Timestamp(end_date))[0][0]

            # Extract the data between these dates
            data = daily_returns_df.iloc[:, start_col:end_col+1]
            data_for_month.append(data)

    # Concatenate all dataframes for the month (if any) and store in the dictionary
    if data_for_month:
        Stocks_MonthlyReturnPastThreeYears_Dataframes[df_name] = pd.concat(data_for_month, axis=1)

In [ ]:
Stocks_MonthlyReturnPastThreeYears_Dataframes["012002"].head(2)

,1999-01-01,1999-02-01,1999-03-01,1999-04-01,1999-05-01,1999-06-01,1999-07-01,1999-08-01,1999-09-01,1999-10-01,...,2001-03-01,2001-04-01,2001-05-01,2001-06-01,2001-07-01,2001-08-01,2001-09-01,2001-10-01,2001-11-01,2001-12-01
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,334.69,369.11,360.81,379.41,417.90,437.52,463.67,497.32,516.88,477.90,...,833.99,778.69,836.62,859.70,786.10,664.19,608.49,456.85,491.28,574.71
ABB LTD N - TOT RETURN IND,1032.11,1021.68,1130.20,1269.54,1450.67,1366.47,1540.00,1549.39,1631.72,1647.76,...,1518.71,1226.94,1432.37,1491.66,1270.67,912.99,761.84,524.29,681.53,796.16


For each MMYY, we now have a dataframe with the monthly average stock prices of each stock in the STOXX 600 in the 3 years prior to MMYY. We will compare each month to the previous week in the dataframe for the relative index. Note that this variable only goes back 2 years instead of three years. Also note that in order to take the relative index of the first month in that period, we also need to have the month before that. We will save the needed dates in *Stocks_MonthlyReturnPastTwoYearsMinus1_Dataframes[MMYYYY]*.

In [ ]:
Stocks_MonthlyReturnPastTwoYearsMinus1_Dataframes = {}

# Define the start and end dates
global_start_date = datetime(2002, 1, 1)
global_end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(global_start_date, global_end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    # Filter the dates DataFrame for the current month and year
    matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    # Initialize a list to collect data for the current month
    for _, row in matching_rows.iterrows():
        # Dates adjusted inside the loop
        calc_start_date = pd.to_datetime(row['StartPastYear']) - pd.DateOffset(years=1, months=1)
        calc_end_date = pd.to_datetime(row['EndPastYear']).replace(day=1)

        # Access the corresponding monthly returns DataFrame
        monthly_returns_df = Stocks_MonthlyReturnPastThreeYears_Dataframes.get(df_name)

        if monthly_returns_df is not None:
            # Assuming columns are labeled by date or a similar method to find columns needs to be adjusted
            try:
                start_col = monthly_returns_df.columns.get_loc(calc_start_date)
                end_col = monthly_returns_df.columns.get_loc(calc_end_date)
                # Extract the data between these dates
                data = monthly_returns_df.iloc[:, start_col:end_col+1]
                Stocks_MonthlyReturnPastTwoYearsMinus1_Dataframes[df_name] = data
            except KeyError:
                print(f"Date columns not found for {df_name} in the DataFrame.")


In [ ]:
Stocks_MonthlyReturnPastTwoYearsMinus1_Dataframes[ "012002"].head(2)

,1999-12-01,2000-01-01,2000-02-01,2000-03-01,2000-04-01,2000-05-01,2000-06-01,2000-07-01,2000-08-01,2000-09-01,...,2001-03-01,2001-04-01,2001-05-01,2001-06-01,2001-07-01,2001-08-01,2001-09-01,2001-10-01,2001-11-01,2001-12-01
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,652.55,731.08,701.19,803.04,911.76,911.88,855.53,900.05,992.19,1172.19,...,833.99,778.69,836.62,859.70,786.10,664.19,608.49,456.85,491.28,574.71
ABB LTD N - TOT RETURN IND,1700.10,2057.39,1919.55,1898.14,2112.00,2112.83,2295.53,2143.20,2219.21,2169.53,...,1518.71,1226.94,1432.37,1491.66,1270.67,912.99,761.84,524.29,681.53,796.16


##### Calculating the long-term momentum

Now we just need to take the percentual difference between the first and the last column

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    if df_name in Stocks_Monthly_Dataframes: #Anders bij bv 3 2024, dit overal toevoegen

         # Extract the relevant slice from the dataframe
        start_values = Stocks_MonthlyReturnPastTwoYearsMinus1_Dataframes[df_name].iloc[:, 0]
        end_values =  Stocks_MonthlyReturnPastTwoYearsMinus1_Dataframes[df_name].iloc[:, -1]

        # Add the 'Monthly LT Momentum' column to the MMYYYY DataFrame in Stock_Monthly_Dataframes
        Stocks_Monthly_Dataframes[df_name]['Momentum_LT'] = ((end_values - start_values) / start_values) * 100

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,IdioVol,Min,Max,Illiquidity,Ln(Turnover),Diff in Ln(Turnover),Momentum_ST,Momentum_MT,Momentum_LT
Name,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,1.846227,-3.322592,5.283979,9.528294e-07,18.533483,-0.185606,1.045432,-28.455038,-11.928588
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,2.576129,-8.358954,6.779872,3.783419e-07,18.920277,-0.386665,-7.212180,-57.891242,-53.169814


#### Monthly Operating Profitability
We add a column *Profitability* to our *Stocks_Monthly_Dataframes['MMYYYY']*. We take the average of the monthly operating profit margin of the stock in the year prior. This is based on the work of Fama and French (2015).

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    if df_name in Stocks_Monthly_Dataframes: #Anders bij bv 3 2024, dit overal toevoegen

        dfOpProf_name = f"{month:02d}{str(year)[2:]}"  # Use slicing to get the last two digits of the year
        data = Stocks_MonthlyOpProf_Dataframes[dfOpProf_name]

        Stocks_Monthly_Dataframes[df_name]['Profitability'] =data.apply(custom_mean, axis=1).values

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,IdioVol,Min,Max,Illiquidity,Ln(Turnover),Diff in Ln(Turnover),Momentum_ST,Momentum_MT,Momentum_LT,Profitability
Name,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,1.846227,-3.322592,5.283979,9.528294e-07,18.533483,-0.185606,1.045432,-28.455038,-11.928588,180833.333333
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,2.576129,-8.358954,6.779872,3.783419e-07,18.920277,-0.386665,-7.212180,-57.891242,-53.169814,569752.666667


#### Monthly Investment
We add a column *Investment* to our *Stocks_Monthly_Dataframes['MMYYYY']*. We take the percentual difference in monthly asset growth in the 13th month prior and the month prior. This is based on the work of Fama and French (2015).

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYYYY
    df_name = f"{month:02d}{year}"

    if df_name in Stocks_Monthly_Dataframes: #Anders bij bv 3 2024, dit overal toevoegen
        # Extract the relevant slice from the dataframe
        dfAssets_name = f"{month:02d}{str(year)[2:]}"  # Use slicing to get the last two digits of the year
        df = Stocks_MonthlyTotalAssets_Dataframes[dfAssets_name]

        # Convert first and last column values to numeric, if not already
        start_values = pd.to_numeric(df.iloc[:, 0], errors='coerce')
        end_values = pd.to_numeric(df.iloc[:, -1], errors='coerce')

        # Calculate the monthly investment growth rate
        growth_rate = ((end_values - start_values) / start_values) * 100

        # Add the 'Monthly Investment' column to the MMYYYY DataFrame
        Stocks_Monthly_Dataframes[df_name]['Investment'] = growth_rate.values

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,...,Min,Max,Illiquidity,Ln(Turnover),Diff in Ln(Turnover),Momentum_ST,Momentum_MT,Momentum_LT,Profitability,Investment
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,...,-3.322592,5.283979,9.528294e-07,18.533483,-0.185606,1.045432,-28.455038,-11.928588,180833.333333,-19.934760
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,...,-8.358954,6.779872,3.783419e-07,18.920277,-0.386665,-7.212180,-57.891242,-53.169814,569752.666667,11.680968


### Fixed Effects

#### Monthly Industry and Exchange Dummies
We create dummy variables in the *Stocks_Monthly_Dataframes['MMYYYY']* for industry and listed stock exchange of the company.

In [ ]:
# Prepare to add dummy variables
for key in Stocks_IndustryExchange_Dataframes:
    key_str = str(key)
    formatted_month = key_str[:2].zfill(2)  # Zero-fill the month to ensure two digits
    formatted_year = key_str[2:]  # Extract the year part (last two digits)
    key_long = f"{formatted_month}20{formatted_year}"
    if key_long in Stocks_Monthly_Dataframes:
        industry_df = Stocks_IndustryExchange_Dataframes[key]
        monthly_df = Stocks_Monthly_Dataframes[key_long]

        # Generate dummy variables for Industry and Exchange
        industry_dummies = pd.get_dummies(industry_df['Industry']).reindex(columns=unique_industries, fill_value=0)
        exchange_dummies = pd.get_dummies(industry_df['Exchange']).reindex(columns=unique_exchanges, fill_value=0)

        # Ensure these dummy columns are boolean
        industry_dummies = industry_dummies.astype(bool)
        exchange_dummies = exchange_dummies.astype(bool)

        # Add prefix to distinguish the columns
        industry_dummies = industry_dummies.add_prefix('Industry_')
        exchange_dummies = exchange_dummies.add_prefix('Exchange_')

        # Append these dummy variables to the monthly DataFrame
        # Use .values to handle potential mismatch in the index
        for col in industry_dummies:
            monthly_df[col] = industry_dummies[col].values
        for col in exchange_dummies:
            monthly_df[col] = exchange_dummies[col].values

        # Save the modified DataFrame back into the Stock_MonthlyDataFrames dictionary
        Stocks_Monthly_Dataframes[key_long] = monthly_df

Streaminguitvoer ingekort tot de laatste 5000 regels.
<ipython-input-117-60850ff85648>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  monthly_df[col] = industry_dummies[col].values
<ipython-input-117-60850ff85648>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  monthly_df[col] = industry_dummies[col].values
<ipython-input-117-60850ff85648>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,...,Exchange_ZU,Exchange_PR,Exchange_LL,Exchange_HL,Exchange_IB,Exchange_PG,Exchange_VN,Exchange_WA,Exchange_AM,Exchange_CP
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,...,False,False,False,False,False,False,False,False,False,False
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,...,True,False,False,False,False,False,False,False,False,False


#### Monthly Size Dummies
We create dummy variables in the *Stocks_Monthly_Dataframes['MMYYYYY']* for market capitalization deciles of the company.

We already have the ln(market cap) of the companies stored under size:

In [ ]:
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,...,Exchange_ZU,Exchange_PR,Exchange_LL,Exchange_HL,Exchange_IB,Exchange_PG,Exchange_VN,Exchange_WA,Exchange_AM,Exchange_CP
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,...,False,False,False,False,False,False,False,False,False,False
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,...,True,False,False,False,False,False,False,False,False,False


We will use this column to create deciles in that month and then give them a rank. And then we can have dummies. The rank of 1 is the lowest and 5 is the highest.

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)
# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    if df_name in Stocks_Monthly_Dataframes:
        df_marketCap = Stocks_Monthly_Dataframes[df_name]['Size']

        # Give each stock in the df_marketCap series a rank from 1 to 5 based on the market capitalization size decile (1 the smallest, 5 the greatest)
        df_marketCap_rank = pd.qcut(df_marketCap, 5, labels=False) + 1

        # Create five dummy variables Size_1, Size_2, ..., Size_5 with boolean values in them,
        # and store the column into the correct dataframe: Stocks_Monthly_Dataframes[df_name]
        for rank in range(1, 6):
            dummy_var_name = f'Size_{rank}'
            Stocks_Monthly_Dataframes[df_name][dummy_var_name] = (df_marketCap_rank == rank).astype(bool)

In [ ]:
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,...,Exchange_CP,Size_1,Size_2,Size_3,Size_4,Size_5,Comove_skippedMonth,Comove_Monthly,Comove_Daily,Comove_EUR50
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-0.104990,15.942634,7.350835,-0.284550,0.701934,0.898882,0.673852,0.553576,...,False,False,False,False,True,False,50.0,41.666667,76.628352,47.169811
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-0.075296,16.281342,9.129183,-0.578912,0.757205,1.305786,0.691230,0.570152,...,False,False,False,False,True,False,62.5,50.000000,73.946360,60.377358


### Monthly Frequency of Comovement with Skipped Month
We add a column *Comove_skippedMonth* to our *Stocks_Monthly_Dataframes['MMYYYY']*. We use the weekly returns of the stock in the last weekday before the prior year starts and the last weekday before the prior month starts. We calculate percentual weekly differences. We do the same for the returns of the stock. Ultimately, we compare the signs of both weekly returns. This is based on the work of Ungeheuer and Weber (2020).

#### Weekly Stock Returns

When calculating the initial comove measure, we made for each MMYY a dataframe with the absolute return indexes of all the stocks in the past 52 weeks before that month. We looked at the year before the month. We started with the first weekday of the previous year (column StartPastYear_Weekday). We ended with the last weekday of the previous year (column EndPastYear_Weekday). We stored them into the Stocks_WeeklyReturn_Dataframes['MMYYYY]. Now, we need to take a slice from this dataframe, that excluded the last month. We look up the EndPastYearExclPastMonth_Weekday column in the dates dataframe to do so.

After that, we will take the weekly average of these dialy abosolute stock returns, just as we did in the initial comovement calculation. Then, we calculate the percentual change and store them in Stocks_WeeklyReturnPercentages_PastYearExclPastMonth_Dataframes[MMYYYY].

In [ ]:
# Stocks_DailyReturn_Dataframes = {'IndexName': dataframe, ...}
# Initialize the dictionary to store the monthly dataframes
Stocks_WeeklyReturnPercentages_PastYearExclPastMonth_Dataframes = {}

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    # Filter the dates DataFrame for the current month and year
    # Assuming 'dates_df' is your DataFrame with 'Month', 'Year', 'Index',
    # 'StartPastYear_Weekday', and 'EndPastYear_Weekday' columns
    matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    # Initialize a list to collect data for the current month
    data_for_month = []

    # If there are dates rows for these periods
    for _, row in matching_rows.iterrows():
        index_name = row['Index']
        startMinus1_date = row['StartPastYearMinus1_Weekday']
        start_date = row['StartPastYear_Weekday']
        end_date = row['EndPastYearExclPastMonth_Weekday']

        # Access the corresponding daily returns DataFrame
        daily_returns_df = Stocks_WeeklyReturn_Dataframes.get(df_name) #dit zijn daily absolute returns, wel verwarrende naam eig

        # If we have return for these period
        if daily_returns_df is not None:

            # Find the columns for start and end dates
            columns = daily_returns_df.columns
            start_col = np.where(columns == pd.Timestamp(start_date))[0][0] # Hier is niet de dag ervoor bij, nog aanpassen
            end_col = np.where(columns == pd.Timestamp(end_date))[0][0]

            # Extract the data between these dates - These are the absolute dialy returns in the correct period
            data = daily_returns_df.iloc[:, start_col:end_col+1]

            # Take the weekly average -  These are the absolute weekly returns in the correct period
            weekly_returns_df = data.resample('W', axis=1).mean()

            # Take the percentual change  - These are the relative weekly returns in the correct period
            weekly_percentage_changes = weekly_returns_df.pct_change(axis=1) * 100

            # The first week will be NaN since we have nothing to compare it with. Let's solve that.
            # Step 1: Look up the absolute daily return on the weekday before the period started
            dataframe = Stocks_DailyReturn_Dataframes.get(index_name)
            day_before_return = dataframe.loc[:, pd.Timestamp(startMinus1_date)] # This is the absolute return of the last weekday before the year starts
            # Step 2: Compare it to the absolute weekly return of the first week
            first_week_return = weekly_returns_df.iloc[:, 0] # This is the absolute return of the first week on average
            first_week_percentage = (( first_week_return - day_before_return ) / day_before_return ) * 100
            # Step 3: Replace the first column with the calculated first week percentage
            weekly_percentage_changes.iloc[:, 0] = first_week_percentage

            # Store the weekly % return
            Stocks_WeeklyReturnPercentages_PastYearExclPastMonth_Dataframes[df_name] = weekly_percentage_changes

In [ ]:
Stocks_WeeklyReturnPercentages_PastYearExclPastMonth_Dataframes["012002"].head(2)

,2001-01-07,2001-01-14,2001-01-21,2001-01-28,2001-02-04,2001-02-11,2001-02-18,2001-02-25,2001-03-04,2001-03-11,...,2001-09-30,2001-10-07,2001-10-14,2001-10-21,2001-10-28,2001-11-04,2001-11-11,2001-11-18,2001-11-25,2001-12-02
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,-1.331504,3.72157,9.815437,2.666851,-0.714534,-5.787722,0.412882,-0.914017,-2.099310,0.314615,...,0.061494,0.597250,8.689161,5.925521,1.434572,-6.933265,7.241564,8.869810,2.685888,-2.422153
ABB LTD N - TOT RETURN IND,-1.647263,0.68384,-3.923507,-0.999026,-0.972986,-2.015261,-10.087911,-5.198335,2.227582,1.289079,...,-3.177462,-5.962853,3.611006,2.762320,17.632136,5.157030,10.128776,8.728026,9.710848,-3.126504


#### Weekly Index Returns

We work almost the same as the initial calculation, only with better names for variables and a different period as well.

In [ ]:
Index_WeeklyReturnPercentages_PastYearExclPastMonth_Dataframes = {}

# Create date range
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)
date_range = pd.date_range(start_date, end_date, freq='MS')  # 'MS' is month start frequency

for single_date in date_range:
    month, year = single_date.month, single_date.year
    df_name = f"{month:02d}{year}"

    # Find the corresponding row in the dates DataFrame
    matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    # If there are corresponding rows
    for _, row in matching_rows.iterrows():
        startMinus1_date = row['StartPastYearMinus1_Weekday']
        start_date = row['StartPastYear_Weekday']
        end_date = row['EndPastYearExclPastMonth_Weekday']

        # Filter the Index_DailyReturn_Dataframe to get rows within the start and end date
        mask = (Index_DailyReturn_Dataframe['Date'] >= start_date) & (Index_DailyReturn_Dataframe['Date'] <= end_date)
        filtered_data = Index_DailyReturn_Dataframe.loc[mask, ['Date', 'Return Index ']]

        # Take the weekly average of the absolute daily indexes
        filtered_data['Date'] = pd.to_datetime(filtered_data['Date']) # Change type of date column so we can use this function that calculates weekly avg
        filtered_data.set_index('Date', inplace=True)
        weekly_avg_df= filtered_data.resample('W').mean()

        # Take the % of weekly averages.
        weekly_avg_df_reset = weekly_avg_df.reset_index()  # Reset index for weekly_returns_df to manipulate 'Date' as a column
        weekly_avg_df_reset['Weekly_Percentage_Change'] = weekly_avg_df_reset['Return Index '].pct_change() * 100

        # The first week will be Nan if you take the percentage now. Let's solve that.
        # Look up the daily index in the date before the period starts.
        dataframe = Index_DailyReturn_Dataframe
        day_before_return = dataframe.loc[dataframe['Date'] == startMinus1_date, 'Return Index '].values[0] # These are rows now not columns
       # Look up the return in the first week
        first_week_return = weekly_avg_df_reset.iloc[0]['Return Index ']
        # Calculate the percentage return for the first week compared to the start date
        first_week_percentage = ((first_week_return - day_before_return) / day_before_return) * 100
        # Store it in the correct place
        weekly_avg_df_reset.loc[0, 'Weekly_Percentage_Change'] = first_week_percentage

        # Store the result in the new dictionary without resetting index
        Index_WeeklyReturnPercentages_PastYearExclPastMonth_Dataframes[df_name] = weekly_avg_df_reset.set_index('Date')[['Weekly_Percentage_Change']]

In [ ]:
Index_WeeklyReturnPercentages_PastYearExclPastMonth_Dataframes["012002"].head(2)

,Weekly_Percentage_Change
Date,
2001-01-07,-0.812000
2001-01-14,-0.921482


#### Comparing Weekly Returns

We use the same code as with the initial calculation, but take our data from different dictionairies

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)
# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    # Check if we have the data for the MMYY in question
    if df_name in Stocks_WeeklyReturnPercentages_PastYearExclPastMonth_Dataframes and df_name in Index_WeeklyReturnPercentages_PastYearExclPastMonth_Dataframes:

        # 1) Get the stock and index dataframes
        stocks_df = Stocks_WeeklyReturnPercentages_PastYearExclPastMonth_Dataframes[df_name]
        index_df = Index_WeeklyReturnPercentages_PastYearExclPastMonth_Dataframes[df_name]

        # 2) Calculate the 'Comove' values
        comove_values = stocks_df.apply(lambda row: calculate_comove(row, index_df.iloc[0]), axis=1) #for each row of the stocks_df (axis=1), apply the function
        # calculate_comove with as first parameter the row itself and as second parameter the first row of the index_df

        # 3) Add to the dictionary
        Stocks_Monthly_Dataframes[df_name]['Comove_skippedMonth'] = comove_values


<ipython-input-126-106d6fe2efd5>:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stocks_Monthly_Dataframes[df_name]['Comove_skippedMonth'] = comove_values
<ipython-input-126-106d6fe2efd5>:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stocks_Monthly_Dataframes[df_name]['Comove_skippedMonth'] = comove_values
<ipython-input-126-106d6fe2efd5>:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

In [ ]:
 Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,...,Exchange_VN,Exchange_WA,Exchange_AM,Exchange_CP,Size_1,Size_2,Size_3,Size_4,Size_5,Comove_skippedMonth
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,...,False,False,False,False,0,0,0,1,0,50.0
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,...,False,False,False,False,0,0,0,1,0,62.5


## Varying the Comove Measure
These are alternative measures for comove (**comove_monthly, comove_weekly, and comove_EUR50**). These are calculated for all stocks in the index in month t, as in Ungeheuer and Weber (2020). This is done for all months. They are stored in *Stocks_Monthly_Dataframes['MMYYYY']*.

### Monthly Comove with Monthly Frequency
We add a column *Comove_Monthly* to our Stocks_Monthly_Dataframes['MMYYYY']. Instead of relying on the last 52 weekly returns from stocks and the index, the analysis incorporates the last 32 monthly returns to calculate Comove.

In [ ]:
# TODO naar bib gaan en die extra maand gaan halen, wel stom, heb nu gewoon effe zonder gedaan.

#### Monthly Stock Returns

##### Monthly Absolute Return Index of Stocks

We have already the the absolute return indexes in the three year prior:

In [ ]:
Stocks_MonthlyReturnPastThreeYears_Dataframes["012002"].head(2)

,1999-01-01,1999-02-01,1999-03-01,1999-04-01,1999-05-01,1999-06-01,1999-07-01,1999-08-01,1999-09-01,1999-10-01,...,2001-03-01,2001-04-01,2001-05-01,2001-06-01,2001-07-01,2001-08-01,2001-09-01,2001-10-01,2001-11-01,2001-12-01
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,334.69,369.11,360.81,379.41,417.90,437.52,463.67,497.32,516.88,477.90,...,833.99,778.69,836.62,859.70,786.10,664.19,608.49,456.85,491.28,574.71
ABB LTD N - TOT RETURN IND,1032.11,1021.68,1130.20,1269.54,1450.67,1366.47,1540.00,1549.39,1631.72,1647.76,...,1518.71,1226.94,1432.37,1491.66,1270.67,912.99,761.84,524.29,681.53,796.16


##### Monthly Relative Return Index of Stocks

In [ ]:
# Placeholder for the new dictionary to store percentage returns DataFrames
Stocks_MonthlyReturnPercentages_Dataframes = {}

for key, monthly_returns_df in Stocks_MonthlyReturnPastThreeYears_Dataframes.items():

    # TODO Calculate the percentage return for the first week compared to the start date

    # Calculate the week-on-week percentage returns for the rest of the weeks
    monthly_percentage_changes = monthly_returns_df.pct_change(axis=1) * 100

    # TODO Replace the first column with the calculated first week percentage

    # Store the result in the new dictionary
    Stocks_MonthlyReturnPercentages_Dataframes[key] = monthly_percentage_changes

In [ ]:
Stocks_MonthlyReturnPercentages_Dataframes["012002"].head(2)

,1999-01-01,1999-02-01,1999-03-01,1999-04-01,1999-05-01,1999-06-01,1999-07-01,1999-08-01,1999-09-01,1999-10-01,...,2001-03-01,2001-04-01,2001-05-01,2001-06-01,2001-07-01,2001-08-01,2001-09-01,2001-10-01,2001-11-01,2001-12-01
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,NaN,10.284144,-2.248652,5.155068,10.144698,4.694903,5.976870,7.257317,3.933081,-7.541402,...,-10.974594,-6.630775,7.439417,2.758720,-8.561126,-15.508205,-8.386155,-24.920705,7.536391,16.982169
ABB LTD N - TOT RETURN IND,NaN,-1.010551,10.621721,12.328791,14.267372,-5.804215,12.699145,0.609740,5.313704,0.983012,...,-15.727881,-19.211699,16.743280,4.139294,-14.815038,-28.148929,-16.555493,-31.181088,29.991035,16.819509


#### Monthly Index Returns

##### Monthly Absolute Return Index of EUR600

In [ ]:
# Assuming Index_MonthlyReturn_Dataframe and dates are defined properly

# Initialize the dictionary to store the DataFrames
Index_MonthlyReturnPastThreeYears_Dataframes = {}

# Create date range
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)
date_range = pd.date_range(start_date, end_date, freq='MS')  # 'MS' is month start frequency

for single_date in date_range:
    month, year = single_date.month, single_date.year
    df_name = f"{month:02d}{year}"

    # Initialize an empty DataFrame for accumulating data for this df_name
    if df_name not in Index_MonthlyReturnPastThreeYears_Dataframes:
        Index_MonthlyReturnPastThreeYears_Dataframes[df_name] = pd.DataFrame()

    # Find the corresponding row in the dates DataFrame
    matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    for _, row in matching_rows.iterrows():
        # This date minus two years
        start_date = pd.to_datetime(row['StartPastYear']) - pd.DateOffset(years=2)
        # This date but set to the 1st of this month
        end_date = pd.to_datetime(row['EndPastYear']).replace(day=1)

        # Filter the Index_DailyReturn_Dataframe to get rows within the start and end date
        mask = (Index_MonthlyReturn_Dataframe['Date'] >= start_date) & (Index_MonthlyReturn_Dataframe['Date'] <= end_date)
        filtered_data = Index_MonthlyReturn_Dataframe.loc[mask, ['Date', 'Return Index']]

        # Append this filtered data to the existing DataFrame in the dictionary
        Index_MonthlyReturnPastThreeYears_Dataframes[df_name] = pd.concat([Index_MonthlyReturnPastThreeYears_Dataframes[df_name], filtered_data])

In [ ]:
 Index_MonthlyReturnPastThreeYears_Dataframes["012002"].head(2) #Note that this will almost all be Nan since they only start the index from 2001

,Date,Return Index
0,1999-01-01,NaN
1,1999-02-01,NaN


##### Monthly Relative Return Index of EUR600

In [ ]:
Index_MonthlyReturnPercentages_Dataframes = {}

for key, monthly_returns_df in Index_MonthlyReturnPastThreeYears_Dataframes.items():
    # Check if the DataFrame is not empty and has at least two columns
    if not monthly_returns_df.empty and monthly_returns_df.shape[1] >= 2:
        # Calculate the week-on-week percentage returns for the rest of the weeks
        monthly_returns_df['Monthly_Percentage_Change'] = monthly_returns_df.iloc[:, 1].pct_change() * 100

        # TODO First week

        # Store the result in the new dictionary
        Index_MonthlyReturnPercentages_Dataframes[key] = monthly_returns_df.set_index('Date')[['Monthly_Percentage_Change']]
    else:
        print(f"DataFrame for {key} is empty or does not have enough columns.")

DataFrame for 032024 is empty or does not have enough columns.


In [ ]:
Index_MonthlyReturnPercentages_Dataframes["012002"].head(2) #Will be Nan for first months since index starts from 2001

,Monthly_Percentage_Change
Date,
1999-01-01,NaN
1999-02-01,NaN


#### Comparing Monthly Returns

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

     # Check if we have the data for the MMYYYY in question
    if df_name in Stocks_MonthlyReturnPercentages_Dataframes and df_name in Index_MonthlyReturnPercentages_Dataframes:

        # 1) Get the stock and index dataframes
        stocks_df = Stocks_MonthlyReturnPercentages_Dataframes[df_name]
        index_df = Index_MonthlyReturnPercentages_Dataframes[df_name]

        # 2) Calculate the 'Comove' values
        comove_values = stocks_df.apply(lambda row: calculate_comove(row, index_df.iloc[0]), axis=1) #for each row of the stocks_df (axis=1), apply the function
        # calculate_comove with as first parameter the row itself and as second parameter the first row of the index_df

        # 3) Write away
        # Add the 'Monthly Comove Montly' column to the MMYYYY DataFrame in Stock_Monthly_Dataframes
        Stocks_Monthly_Dataframes[df_name]['Comove_Monthly'] =  comove_values

<ipython-input-136-7d69caa01c1b>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stocks_Monthly_Dataframes[df_name]['Comove_Monthly'] =  comove_values
<ipython-input-136-7d69caa01c1b>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stocks_Monthly_Dataframes[df_name]['Comove_Monthly'] =  comove_values
<ipython-input-136-7d69caa01c1b>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,...,Exchange_WA,Exchange_AM,Exchange_CP,Size_1,Size_2,Size_3,Size_4,Size_5,Comove_skippedMonth,Comove_Monthly
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,...,False,False,False,0,0,0,1,0,50.0,41.666667
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,...,False,False,False,0,0,0,1,0,62.5,50.000000


### Monthly Comove with Daily Frequency
We add a column *Comove_Daily* to our Stocks_Monthly_Dataframes['MMYYYY']. Instead of relying on the last 52 weekly returns from stocks and the index, the analysis incorporates the last 260 daily returns to calculate Comove.

Since we already needed the daily return percentages in the year prior for the calculation of beta, we have them already stored:

In [ ]:
Stocks_DailyReturnPastYearPercentages_Dataframes["022002"].head(2)

,2001-02-01,2001-02-02,2001-02-05,2001-02-06,2001-02-07,2001-02-08,2001-02-09,2001-02-12,2001-02-13,2001-02-14,...,2002-01-18,2002-01-21,2002-01-22,2002-01-23,2002-01-24,2002-01-25,2002-01-28,2002-01-29,2002-01-30,2002-01-31
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,-1.374939,-0.974594,-2.176420,0.740512,-3.384380,0.228698,-3.516402,1.479834,3.239539,-2.171265,...,-3.071834,0.439257,0.088552,-4.627690,0.064368,1.992243,-1.376419,-1.158635,-0.298763,1.708243
ABB LTD N - TOT RETURN IND,-1.948889,0.841217,0.110603,0.573841,-1.303996,-2.892740,-0.965980,1.092865,-8.496896,-4.175485,...,-3.703658,-0.446983,-1.882156,-3.060301,3.313811,4.709163,0.836459,-1.146945,-7.014199,-4.594507


In [ ]:
Index_DailyReturnPastYearPercentages_Dataframes["012002"].head(2)

,Daily_Percentage_Change
Date,
2001-01-01,0.00
2001-01-02,-0.91


We can now just use our function to calculate the comove:

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:

   # Find dataframe names we need, based on the year
    df_name = str(period.month).zfill(2) + str(period.year)

    # Check if we have the data for the year in question
    if df_name in Stocks_DailyReturnPastYearPercentages_Dataframes and df_name in Index_DailyReturnPastYearPercentages_Dataframes:

        # 1) Get the stock and index dataframes
        stocks_df = Stocks_DailyReturnPastYearPercentages_Dataframes[df_name]
        index_df = Index_DailyReturnPastYearPercentages_Dataframes[df_name]

        # 2) Calculate the beta values
        beta_values = stocks_df.apply(lambda row: calculate_comove(row, index_df.iloc[:,0]), axis=1) #for each row of the stocks_df (axis=1), apply the function
        # calculate_comove with as first parameter the row itself and as second parameter the first column of the index_df

        # 3) Add this columnn to the comove values we already had
        Stocks_Monthly_Dataframes[df_name]['Comove_Daily'] = beta_values

<ipython-input-140-c830549880c0>:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stocks_Monthly_Dataframes[df_name]['Comove_Daily'] = beta_values
<ipython-input-140-c830549880c0>:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stocks_Monthly_Dataframes[df_name]['Comove_Daily'] = beta_values
<ipython-input-140-c830549880c0>:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,...,Exchange_AM,Exchange_CP,Size_1,Size_2,Size_3,Size_4,Size_5,Comove_skippedMonth,Comove_Monthly,Comove_Daily
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,...,False,False,0,0,0,1,0,50.0,41.666667,76.628352
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,...,False,False,0,0,0,1,0,62.5,50.000000,73.946360


### Monthly Comove with EUR50
We add a column *Comove_EUR50* to our Stocks_Monthly_Dataframes['MMYYYY']. The weekly returns of stocks are now compared against the EUR50 index rather than the EUR600.

#### Weekly Stock Returns

There were already calculated when we initially looked at comove:

In [ ]:
# Now, Stocks_WeeklyReturnPercentages_Dataframes contains the week-on-week percentage returns for each month/year,
# including the first week's percentage calculated from the StartPastYearMinus1_Weekday.
Stocks_WeeklyReturnPercentages_Dataframes["012002"].head(2)

,2001-01-07,2001-01-14,2001-01-21,2001-01-28,2001-02-04,2001-02-11,2001-02-18,2001-02-25,2001-03-04,2001-03-11,...,2001-11-04,2001-11-11,2001-11-18,2001-11-25,2001-12-02,2001-12-09,2001-12-16,2001-12-23,2001-12-30,2002-01-06
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,-1.331504,3.72157,9.815437,2.666851,-0.714534,-5.787722,0.412882,-0.914017,-2.099310,0.314615,...,-6.933265,7.241564,8.869810,2.685888,-2.422153,1.414722,-3.685823,0.278520,1.626058,1.194847
ABB LTD N - TOT RETURN IND,-1.647263,0.68384,-3.923507,-0.999026,-0.972986,-2.015261,-10.087911,-5.198335,2.227582,1.289079,...,5.157030,10.128776,8.728026,9.710848,-3.126504,1.154862,-6.706429,-6.289987,0.418242,0.428843


#### Weekly Index EUR50 Returns

##### Weekly Absolute Return Index of EUR50

We follow the same staps as we did with EUR600:

In [ ]:
# Initialize the dictionary to store the DataFrames
IndexEUR50_WeeklyReturn_Dataframes = {}

# Create date range
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)
date_range = pd.date_range(start_date, end_date, freq='MS')  # 'MS' is month start frequency

for single_date in date_range:
    month, year = single_date.month, single_date.year
    df_name = f"{month:02d}{year}"

    # Find the corresponding row in the dates DataFrame
    matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    for _, row in matching_rows.iterrows():
        index_name = row['Index']  # Though you mentioned Index, it seems like it's not used directly here.
        start_date = row['StartPastYear_Weekday']
        end_date = row['EndPastYear_Weekday']

        # Filter the Index_DailyReturn_Dataframe to get rows within the start and end date
        mask = (IndexEUR50_DailyReturn_Dataframe['Date'] >= start_date) & (IndexEUR50_DailyReturn_Dataframe['Date'] <= end_date)
        filtered_data = IndexEUR50_DailyReturn_Dataframe.loc[mask, ['Date', 'Return Index']]

        # Store this filtered data in the Index_WeeklyReturn_Dataframes dictionary
        IndexEUR50_WeeklyReturn_Dataframes[df_name] = filtered_data

In [ ]:
IndexEUR50_WeeklyReturn_Dataframes["012002"].head(2)

,Date,Return Index
21,2001-01-01,NaN
22,2001-01-02,985.3


Now we need to calculate the average per week. Note that the year not always starts on a monday and ends on a friday, so we do not always have five daily returns to calculate the weekly ones. We store these in Index_WeeklyReturnAverages_Dataframes['MMYYYY'].

In [ ]:
IndexEUR50_WeeklyReturnAverages_Dataframes = {}

for key, df in IndexEUR50_WeeklyReturn_Dataframes.items():
     # Ensure the 'Date' column is of datetime type and set it as the index
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Now that 'Date' is the index, resample the DataFrame to get weekly averages.
    # Note: This assumes that you want to calculate the mean of the 'Return' column on a weekly basis.
    # If your DataFrame contains multiple columns you want to average, you might need to adjust this.
    weekly_avg_df = df.resample('W').mean()

    # Store the resulting DataFrame of weekly averages in the new dictionary
    IndexEUR50_WeeklyReturnAverages_Dataframes[key] = weekly_avg_df


In [ ]:
IndexEUR50_WeeklyReturnAverages_Dataframes["012002"].head(2)

,Return Index
Date,
2001-01-07,987.53
2001-01-14,979.83


##### Weekly Relative Return Index of EUR50

For each MMYY, we now have a dataframe with the weekly average return index of the STOXX 50 in the year prior to MMYY. We will compare each week to the previous week in the dataframe. The first week will be compared to the last weekday before the previous year period starts. We will calculate percent of change in total return index. These will be saved in IndexEUR50_WeeklyReturnPercentages_Dataframes['MMYY'].

In [ ]:
IndexEUR50_WeeklyReturnPercentages_Dataframes = {}

for key, weekly_returns_df in IndexEUR50_WeeklyReturnAverages_Dataframes.items():
    # Convert key to month and year
    month, year = int(key[:2]), int(key[2:])
    # Find the corresponding information in the dates DataFrame
    date_info = dates[(dates['Month'] == month) & (dates['Year'] == year)]

    if not date_info.empty:
        for _, row in date_info.iterrows():
            index_name = row['Index']
            start_past_year_date = pd.to_datetime(row['StartPastYearMinus1_Weekday'])

            # Access the daily return on the start date from the corresponding DataFrame
            daily_return_df = IndexEUR50_DailyReturn_Dataframe

            if daily_return_df is not None and start_past_year_date in daily_return_df['Date'].values:
                # Reset index if 'Date' is the index column
                daily_return_df_reset = daily_return_df.reset_index()
                start_return = daily_return_df_reset.loc[daily_return_df_reset['Date'] == start_past_year_date, 'Return Index'].values[0]

                # Reset index for weekly_returns_df to manipulate 'Date' as a column
                weekly_returns_df_reset = weekly_returns_df.reset_index()
                first_week_return = weekly_returns_df_reset.iloc[0]['Return Index']
                # Calculate the percentage return for the first week compared to the start date
                first_week_percentage = ((first_week_return - start_return) / start_return) * 100

                # Calculate the week-on-week percentage returns for the rest of the weeks
                weekly_returns_df_reset['Weekly_Percentage_Change'] = weekly_returns_df_reset['Return Index'].pct_change() * 100
                # Replace the first percentage change with the calculated first week percentage
                weekly_returns_df_reset.loc[0, 'Weekly_Percentage_Change'] = first_week_percentage

                # Store the result in the new dictionary without resetting index
                IndexEUR50_WeeklyReturnPercentages_Dataframes[key] = weekly_returns_df_reset.set_index('Date')[['Weekly_Percentage_Change']]

In [ ]:
IndexEUR50_WeeklyReturnPercentages_Dataframes["012002"].head(2)

,Weekly_Percentage_Change
Date,
2001-01-07,NaN
2001-01-14,-0.779723


#### Comparing Weekly Returns

In [ ]:
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYYYY
    df_name = f"{month:02d}{year}"

    if df_name in Stocks_Monthly_Dataframes: #Anders bij bv 3 2024, dit overal toevoegen

         # 1) Get the stock and index dataframes
        stocks_df = Stocks_WeeklyReturnPercentages_Dataframes[df_name]
        index_df = IndexEUR50_WeeklyReturnPercentages_Dataframes[df_name]

        # 2) Calculate the 'Comove' values
        comove_values = stocks_df.apply(lambda row: calculate_comove(row, index_df.iloc[0]), axis=1) #for each row of the stocks_df (axis=1), apply the function
        # calculate_comove with as first parameter the row itself and as second parameter the first row of the index_df

        # 3) Add the 'Monthly Comove EUR50' column to the MMYYYY DataFrame in Stock_Monthly_Dataframes
        Stocks_Monthly_Dataframes[df_name]['Comove_EUR50'] =  comove_values.values

<ipython-input-149-5720fd30bc23>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stocks_Monthly_Dataframes[df_name]['Comove_EUR50'] =  comove_values.values
<ipython-input-149-5720fd30bc23>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stocks_Monthly_Dataframes[df_name]['Comove_EUR50'] =  comove_values.values
<ipython-input-149-5720fd30bc23>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at on

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,...,Exchange_CP,Size_1,Size_2,Size_3,Size_4,Size_5,Comove_skippedMonth,Comove_Monthly,Comove_Daily,Comove_EUR50
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-10.499001,15.942634,7.350835,-28.455038,0.701934,0.898882,0.673852,0.553576,...,False,0,0,0,1,0,50.0,41.666667,76.628352,47.169811
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-7.529581,16.281342,9.129183,-57.891242,0.757205,1.305786,0.691230,0.570152,...,False,0,0,0,1,0,62.5,50.000000,73.946360,60.377358


## Writing the Full Data for the Statistical Tests to Excel
We prepare the data for writing. We write the all the data into of *Stocks_Monthly_Dataframes* into an excel file *Data_Monthly.xlsx* where each sheets represents a MMYYYY.

In [ ]:
# TODO Extra: Zou die comove ook moeten gedeeld worden door 100?

First, we need to prepare the variables for regression. We will divide all return columns by 100. Total asset is also divided by 100 since this is a growth rate as well.

In [ ]:
columns = ['Return', 'Momentum', 'Min', 'Max', 'Momentum_ST', 'Momentum_MT', 'Momentum_LT', 'Investment']
# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31) # DEBUG end_date = datetime(2002, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    # Add the 'Monthly Size' column to the MMYYYY DataFrame in Stock_Monthly_Dataframes
    df = Stocks_Monthly_Dataframes.get(df_name)

    # I want to write to an excel Data_Monthly that has a sheet MMYYYY with this dataframe.
    if df is not None:
        # Ensure the columns exist in the DataFrame to avoid KeyError
        columns_to_modify = [col for col in columns if col in df.columns]
        # Divide the selected columns by 100
        df[columns_to_modify] = df[columns_to_modify].apply(lambda x: x / 100)

In [ ]:
# This code would result in the Monthly_Dataframes dictionary being populated with a dataframe for each month from 2002 to 2024.
Stocks_Monthly_Dataframes["012002"].head(2)

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,...,Exchange_CP,Size_1,Size_2,Size_3,Size_4,Size_5,Comove_skippedMonth,Comove_Monthly,Comove_Daily,Comove_EUR50
Name,,,,,,,,,,,,,,,,,,,,,
3I GROUP - TOT RETURN IND,47.169811,1.405906,-0.104990,15.942634,7.350835,-0.284550,0.701934,0.898882,0.673852,0.553576,...,False,0,0,0,1,0,50.0,41.666667,76.628352,47.169811
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-0.075296,16.281342,9.129183,-0.578912,0.757205,1.305786,0.691230,0.570152,...,False,0,0,0,1,0,62.5,50.000000,73.946360,60.377358


Then we can start writing to Excel:

In [ ]:
# Define the file path
file_path = 'Data_Monthly.xlsx'

# Initialize the Excel writer with appropriate mode based on file existence
excel_writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='w')

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31) # DEBUG  end_date = datetime(2002, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    # Add the 'Monthly Size' column to the MMYYYY DataFrame in Stock_Monthly_Dataframes
    df = Stocks_Monthly_Dataframes.get(df_name)

    # I want to write to an excel Data_Monthly that has a sheet MMYYYY with this dataframe.
    if df is not None:
       # Write the DataFrame to an Excel sheet named MMYYYYY
        sheet_name = df_name
        try:
            df.to_excel(excel_writer, sheet_name=sheet_name)
        except Exception as e:
            print(f"Failed to write {sheet_name} to Excel: {e}")
    else:
        # Optionally, log or handle the case where the DataFrame does not exist
        print(f"No data available for {df_name}")

# Make sure to save and close the writer to commit changes to the file
excel_writer.close()

No data available for 032024


## Varying the Sample data
We exclude certain stocks in one alternative sample and write them away to *DataExcl.xlsx*. The sample split is done in the model phasing to avoid having all the data stored on disk twice.

In [ ]:
# Dit moet naar aparte excels
# Excluding NASDAQ stocks, including small firms below the 1st NYSE-decile or excluding stocks with end-of-last-month prices below $5

### Excluding stocks from the Londen Stock Exchange, small companies and stocks with low prices
We exclude stocks that are from the Londen  Stock (LN) exchange, that are in the lowest size decile based on market capitalization, and that had an absolute return index below 5 euro at the end of the month. The result is stored in the *DataExcl_Monthly.xlsx* file.

There are 265 months as datapoints. There are 600 stocks in each. In total, there are 159.000 individual stocks. As measured when reading in the exchange data, 45852 are from the Londen stock exchange. The next most common stock exchange in line is the Euronext Paris stock exchange which only has 21399 stocks, which is about half.  Here are the ten most common ones:

[('LN', 45852), ('PR', 21399), ('IB', 17197), ('ZU', 12275), ('ST', 11166), ('ML', 9448), ('MC', 8250), ('AM', 8071), ('CP', 4967), ('HL', 4605)]

In [ ]:
# Initialize the dictionary to store filtered monthly dataframes
StocksExcl_Monthly_Dataframes = {}

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)  # DEBUG end_date = datetime(2002, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYYYY
    df_name = f"{month:02d}{year}"

    if df_name in Stocks_Monthly_Dataframes:
        df = Stocks_Monthly_Dataframes[df_name]

        # Get the rows for the current month and year from the dates DataFrame
        matching_rows = dates[(dates['Month'] == month) & (dates['Year'] == year)]
        data_for_month = []
        for _, row in matching_rows.iterrows():
            index_name = row['Index']
            end_date = row['EndMonth_Weekday']
            # Access the corresponding daily returns DataFrame
            daily_returns_df = Stocks_DailyReturn_Dataframes.get(index_name)
            if daily_returns_df is not None:
                # Find the column corresponding to the end date of the month
                columns = daily_returns_df.columns
                try:
                    end_col_idx = np.where(columns == pd.Timestamp(end_date))[0][0]
                    # Extract the data on the last day of the month
                    data = daily_returns_df.iloc[:, end_col_idx]
                    data_for_month.append(data)
                except IndexError:
                    print(f"End date {end_date} not found in columns")  # Debug statement
        if data_for_month:
            data = data_for_month[0]
        # Exclude rows in the df that have return < 5, be aware that index is not the same but order is
        df = df[data >= 5]

        # If the column "Exchange_LN" is True, exclude the row from df
        if 'Exchange_LN' in df.columns:
            df = df[df['Exchange_LN'] != True]

        # Exclude rows where the column "Size_1" is True
        if 'Size_1' in df.columns:
            df = df[df['Size_1'] != True]

        # Write data away
        StocksExcl_Monthly_Dataframes[df_name] = df
    else:
        print(f"No data for {df_name}.\n")  # Debug statement


No data for 032024.



In [ ]:
StocksExcl_Monthly_Dataframes["012002"].head(2) #331 rows of the 600 remain in this month

,Comove,Beta,Return,Size,Value,Momentum,Beta_Downside,Beta_Upside,LTD,UTD,...,Exchange_CP,Size_1,Size_2,Size_3,Size_4,Size_5,Comove_skippedMonth,Comove_Monthly,Comove_Daily,Comove_EUR50
Name,,,,,,,,,,,,,,,,,,,,,
ABB LTD N - TOT RETURN IND,60.377358,1.675130,-0.075296,16.281342,9.129183,-0.578912,0.757205,1.305786,0.691230,0.570152,...,False,0,0,0,1,0,62.500000,50.000000,73.946360,60.377358
ABN AMRO HOLDING DEAD - 25/04/08 - TOT RETURN IND,54.716981,1.317305,0.112780,17.139726,NaN,-0.215650,1.022924,0.820010,0.714905,0.593645,...,False,0,0,0,0,1,54.166667,44.444444,74.712644,54.716981


We will write this away to excel:

In [ ]:
# Define the file path
file_path = 'DataExcl_Monthly.xlsx'

# Initialize the Excel writer with appropriate mode based on file existence
excel_writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='w')

# Define the start and end dates
start_date = datetime(2002, 1, 1)
end_date = datetime(2024, 3, 31)  # DEBUG end_date = datetime(2002, 3, 31)

# Generate the monthly periods
monthly_periods = pd.date_range(start_date, end_date, freq='M')

for period in monthly_periods:
    # Extract the year and month
    year = period.year
    month = period.month

    # Format the DataFrame name as MMYY
    df_name = f"{month:02d}{year}"

    # Add the 'Monthly Size' column to the MMYYYY DataFrame in Stock_Monthly_Dataframes
    df = StocksExcl_Monthly_Dataframes.get(df_name)

    # I want to write to an excel Data_Monthly that has a sheet MMYYYY with this dataframe.
    if df is not None:
       # Write the DataFrame to an Excel sheet named MMYYYYY
        sheet_name = df_name
        try:
            df.to_excel(excel_writer, sheet_name=sheet_name)
        except Exception as e:
            print(f"Failed to write {sheet_name} to Excel: {e}")
    else:
        # Optionally, log or handle the case where the DataFrame does not exist
        print(f"No data available for {df_name}")

# Make sure to save and close the writer to commit changes to the file
excel_writer.close()

No data available for 032024


### Sample Split
When testing the data, we will simple pick the correct months in the from the full sample Excel. Otherwise we have to save the big data file on disk twice.

## References
A bibliography of the cited literature.

Amihud, Y. (2002). Illiquidity and stock returns: Cross-section and time-series effects. *Journal of Financial Markets*, 5(1), 31–56. https://doi.org/10.1016/S1386-4181(01)00024-6

Bali, T. G., Cakici, N., & Whitelaw, R. F. (2011). Maxing out: Stocks as lotteries and the cross-section of expected returns. *Journal of Financial Economics*, 99(2), 427–446. https://doi.org/10.1016/j.jfineco.2010.08.014

Chabi-Yo, F., Ruenzi, S., & Weigert, F. (2017). *Crash Sensitivity and the Cross-Section of Expected Stock Returns* (SSRN Scholarly Paper 2011746). https://doi.org/10.2139/ssrn.2011746

Chen, J., Ang, A., & Xing, Y. (2005). *Downside Risk* (SSRN Scholarly Paper 875700). https://papers.ssrn.com/abstract=875700

Fama, E. F., & French, K. R. (1992). The Cross-Section of Expected Stock Returns. *The Journal of Finance*, 47(2), 427–465. https://doi.org/10.1111/j.1540-6261.1992.tb04398.x

Fama, E. F., & MacBeth, J. D. (1973). Risk, Return, and Equilibrium: Empirical Tests. *Journal of Political Economy*, 81(3), 607–636.


Gervais, S., Kaniel, R., & Mingelgrin, D. H. (2001). The High-Volume Return Premium. *The Journal of Finance*, 56(3), 877–919. https://doi.org/10.1111/0022-1082.00349

Ungeheuer, M., & Weber, M. (2020). *The Perception of Dependence, Investment Decisions, and Stock Prices* (SSRN Scholarly Paper 2739130). https://doi.org/10.2139/ssrn.2739130